# Adaptive PDE discretizations on Cartesian grids
## Volume : Divergence form PDEs
## Part : Acoustic and elastic waves
## Chapter : Staggered grid schemes

$
\newcommand\cE{\mathcal{E}}
\newcommand\<{\langle} \newcommand\>{\rangle}
\newcommand\be{\boldsymbol{e}}
\newcommand\kw{\mathrm{k}}
\newcommand\ri{\mathrm{i}}
\newcommand\rI{\mathrm{I}}
\newcommand\diff{\mathrm{d}}
\newcommand\cO{\mathcal{O}}
\DeclareMathOperator\Tr{Tr}
\newcommand\Id{\mathrm{Id}}
\newcommand\bR{\mathbb{R}}
\newcommand\bZ{\mathbb{Z}}
\newcommand\modtwo{[\text{mod}\, 2]}
$

In a series of other notebooks, we introduced a discretization of the elastic pootential energy, and thus a Selling/Voronoi decomposition based numerical scheme for the elastic wave equation, see [the elastic energy](ElasticEnergy), [wave propagation](ElasticWave), [high order schemes](HighOrderWaves), [gradient backpropagation](WaveExamples). The objective of this notebook is to compare the introduced method, referred to as the Selling elastic scheme, to existing methods in the literature. More precisely we consider :

- the Virieux scheme, which assumes a VTI Hooke tensor
- the Lebedev scheme, which is defined as a collection of $2^{d-1}$ coupled Virieux schemes, and allows any Hooke tensor.

The Lebedev scheme and the proposed Selling scheme, which both allow arbitrary Hooke tensors, can be compared on the following points. 
- *energy conservation*. The Selling scheme preserves a perturbed energy exactly, and therefore preserves the target energy up to a multiplicative constant (close to one if the CFL is satisfied). The Virieux and Lebedev schemes do not have this guarantee, and we observe strong oscillations in the energy (with random Hooke tensors).
- *grid decoupling*. The Selling scheme preserves a coercive energy, which controls the solution regularity via the Korn inequality, and forbids e.g chessboard artifacts. The Lebedev scheme does not offer such a guarantee, and in fact the solution on two subgrids may decouple and evolve independently in a chessboard pattern.
- *numerical dispersion relation*. For a given number of discretization points per wavelength, which scheme has the most accurate numerical dispersion relation ? We observe that the Virieux scheme is the most accurate for VTI models, followed by the proposed Selling scheme, and last the Lebedev scheme. For non-VTI models (specifically, the mica medium), the Virieux scheme is not applicable, the situation is more nuanced : the proposed Selling scheme appears best for the pressure wave, and the Lebedev scheme for the shear wave (a proposed optimization may cure this issue). 

We implement the Virieux and Lebedev schemes, but only in a quick and dirty way which is only meant to perform basic comparisons and check the dispersion relations.

**Acoustic waves.**
Their case is much simpler, and discussed in the last section of the notebook. It is in the acoustic (scalar) setting that the Selling schemes are the most efficient, with the best dispersion.

[**Summary**](Summary.ipynb) of volume Divergence form PDEs, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1. Staggered grids](#1.-Staggered-grids)
  * [2. The Virieux scheme](#2.-The-Virieux-scheme)
    * [2.1 Standard finite differences implementation](#2.1-Standard-finite-differences-implementation)
    * [2.2 As a symplectic scheme](#2.2-As-a-symplectic-scheme)
  * [3. The Lebedev scheme](#3.-The-Lebedev-scheme)
    * [3.1 Finite differences implementation](#3.1-Finite-differences-implementation)
    * [3.2 Symplectic implementation](#3.2-Symplectic-implementation)
  * [4. Dispersion relations](#4.-Dispersion-relations)
  * [5. Numerical dispersion](#5.-Numerical-dispersion)
    * [5.1 Isotropic dispersion](#5.1-Isotropic-dispersion)
    * [5.2 VTI and TTI dispersion](#5.2-VTI-and-TTI-dispersion)
  * [6. Correlated Selling scheme](#6.-Correlated-Selling-scheme)
    * [6.1 Scheme implementation](#6.1-Scheme-implementation)
    * [6.2 Dispersion relation](#6.2-Dispersion-relation)
    * [6.3 Visualizing dispersion](#6.3-Visualizing-dispersion)
    * [6.4 Three dimensional case](#6.4-Three-dimensional-case)
  * [7. Staggered Selling scheme](#7.-Staggered-Selling-scheme)
    * [7.1 Staggered Selling decomposition](#7.1-Staggered-Selling-decomposition)
    * [7.2 Scheme implementation](#7.2-Scheme-implementation)
    * [7.3 Dispersion relation](#7.3-Dispersion-relation)
    * [7.4 Comparing dispersions](#7.4-Comparing-dispersions)
    * [7.5 Yet another variant](#7.5-Yet-another-variant)
  * [8. Energy conservation](#8.-Energy-conservation)
  * [9. Grid decoupling in the Lebedev scheme](#9.-Grid-decoupling-in-the-Lebedev-scheme)
  * [10. Acoustic scheme](#10.-Acoustic-scheme)
    * [10.1 Centered scheme](#10.1-Centered-scheme)
    * [10.2 Criss-cross scheme](#10.2-Criss-cross-scheme)
    * [10.3 Dispersion](#10.3-Dispersion)
  * [11. Image and animation exports](#11.-Image-and-animation-exports)
    * [11.1 Acoustic](#11.1-Acoustic)
    * [11.2 Elastic](#11.2-Elastic)



**Acknowledgement.** Some of the experiments presented in these notebooks are part of 
ongoing research with Ludovic Métivier and Da Chen.

Copyright Jean-Marie Mirebeau, Centre Borelli, ENS Paris-Saclay, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; print(TocTools.displayTOC('ElasticComparisons','Div'))

In [2]:
from agd.Metrics.Seismic import Hooke
from agd.Metrics import Riemann
from agd import FiniteDifferences as fd
from agd import LinearParallel as lp
from agd.ExportedCode.Notebooks_Div.HighOrderWaves import make_domain,dispersion_e,dispersion_a,MakeRandomTensor
from agd.Eikonal.HFM_CUDA import AnisotropicWave
from agd.ODE.hamiltonian import QuadraticHamiltonian
from agd import AutomaticDifferentiation as ad
from agd.Metrics import Seismic
from agd.Metrics.misc import flatten_symmetric_matrix
from agd.Plotting import savefig; #savefig.dirName = 'Images/ElasticComparisons/'

SyntaxError: 'function call' is an illegal expression for augmented assignment (hamiltonian.py, line 334)

In [3]:
import numpy as np
import itertools
import copy
from scipy.optimize import linprog
π = np.pi

In [4]:
from matplotlib import pyplot as plt
from matplotlib.patches import Patch
from matplotlib import rc,animation; rc('animation', html='html5')
np.set_printoptions(linewidth=2000)

## 1. Staggered grids

The Virieux and Lebedev schemes rely on Staggered grids : in two dimensions, the scheme unknowns may be stored at 
\begin{align*}
    & (i,j) \diff x &
    & (i+1/2,j) \diff x &
    & (i,j+1/2) \diff x &
    & (i+1/2,j+1/2) \diff x,
\end{align*}
where $i,j \in \bZ$. This is emphasized by subscripts `_00`, `_10`, `_01`, `_11`, in the variable name. The proposed Selling scheme, in contrast, uses the standard grid.

In [5]:
def shifted_grids(X,dx):
    """
    Shifted grids for staggered scheme.
    X_0,X_1
    X_00,X_10,X_01,X_11 (inverted binary order)
    X_000,X_100,X_010,X_110,X_001,X_101,X_011,X_111
    """
    vdim = len(X)
    shape = X[0].shape
    shifts = itertools.product((0,dx/2),repeat=len(X))
    return [X + fd.as_field(s[::-1],shape) for s in shifts]

The point of the staggered grid representation is to have second order accurate finite differences for derivatives along suitable grid axes. 
For instance
$$
    \frac{\partial}{\partial x} u( (i,j) \diff x) = \frac{u( (i+1/2,j) \diff x) - u( (i-1/2,j) \diff x)}{\diff x} + \cO(\diff x^2).
$$
Fourth order accurate finite differences likewise have a small four point stencil.

In [6]:
class staggered:
    """A class for first order finite difference operators on staggered grids"""
    
    def __init__(self,dx,order=2,bc='Periodic'):
        self.idx=1/dx
        self.order=order; assert order in (2,4)
        self.bc=bc; assert bc in ('Periodic','Dirichlet')

    def roll(self,q,shift,axis,bc=None):
        """Rolls specified axis, inserts zeros in case of Dirichlet boundary conditions"""
        if bc is None: bc=self.bc
        q = np.roll(q,shift,axis)
        if bc!='Periodic': 
            pad = q[*(slice(None),)*axis,shift,None] if bc=='Constant' else 0.
            if shift>0: q[*(slice(None),)*axis,:shift]=pad
            else: q[*(slice(None),)*axis,shift:]=pad
        return q
        
    def diff_left(self,q,axis,s=0): 
        """
        First order finite difference, to the left, along axis i.
        (Result is shifted by half a grid step to the left.)
        """
        dq = self.roll(q,-s,axis)-self.roll(q,1-s,axis) # Centered finite difference, with step 1/2
        if self.order==2: return dq * self.idx 
        dq2 = self.roll(q,-1-s,axis)-self.roll(q,2-s,axis) # Centered finite difference, with step 3/2
        return ( (9/8)*dq-(1/24)*dq2 )*self.idx # Fourth order finite difference
    
    def diff_right(self,q,axis):
        """
        First order finite difference, to the right, along axis i.
        (Result is shifted by half a grid step to the right.)
        """
        return self.diff_left(q,axis,s=1)

    def avg_left(self,q,axis,s=0,bc=None):
        """
        Approximate value, half a grid step to the left, along axis i.
        (Use bc='Constant' for averaging coefficients with Dirichlet boundary conditions.)
        """
        if isinstance(axis,tuple): # Average over several axes
            for ax in axis: q = self.avg_left(q,ax,s,bc)
            return q
        if bc is None: bc = self.bc
        aq = self.roll(q,-s,axis,bc)+self.roll(q,1-s,axis,bc) # Centered average, with step 1/2
        if self.order==2: return aq * 0.5 
        aq2 = self.roll(q,-1-s,axis,bc)+self.roll(q,2-s,axis,bc) # Centered average, with step 3/2
        return (9/16)*aq - (1/16)*aq2 # Fourth order accurate interpolation 
    
    def avg_right(self,q,axis):
        return self.avg_left(q,axis,s=1)

    def diff_left_offset(self,q,axis,offset,right=False):
        """
        First order finite difference, along given offset. Shifts result by 1/2 along axis i to the left.
        - right (bool, optional) : shift to the right instead
        Assumption : offset[axis] must be odd.
        Example : diff_left_offset(q,i,eye[i]) == diff_left(q,i)
        """
        e_left = np.array(offset).astype(int)//2
        e_right = e_left.copy() 
        if np.ndim(right)==0: e_left[axis]+=1-right; e_right[axis]+=right
        else: e_left[axis][np.logical_not(right)]+=1; e_right[axis][right]+=1
        assert np.all(offset==e_left+e_right)
        vdim = len(offset)
        ax = tuple(range(vdim))
        dq = self.roll(q,-e_right,ax) - self.roll(q,e_left,ax)
        if self.order==2: return dq * self.idx # Second order finite difference
        dq2 = self.roll(q,-e_right-offset,ax) - self.roll(q,e_left+offset,ax)
        return ( (9/8)*dq-(1/24)*dq2 )*self.idx # Fourth order finite difference

    def diff_right_offset(self,q,axis,offset):
        """
        Assumption : offset[axis] must be odd.
        Example : diff_right_offset(q,i,eye[i]) == diff_right(q,i)
        """
        return self.diff_left_offset(q,axis,offset,right=True)

## 2. The Virieux scheme

This scheme assumes a VTI hooke tensor. In two dimensions, the structure reads
$$
    C = \begin{pmatrix}
    * & * & \\
    * & * & \\
      &   & *
    \end{pmatrix}.
$$
In other words, $C_{02} = C_{12} = 0$. (And likewise $C_{20}=C_{21}=0$ by symmetry.)

The Virieux scheme uses a staggered grid, with the following variable locations : 
- the *position and momentum* components are stored in shifted positions along their respective axis. For instance, the unknowns associated to the position $q=(q_0,q_1)$ are for $i,j \in \bZ$
\begin{align*}
    & q_0( (i+1/2,j) \diff x), &
    & q_1( (i,j+1/2) \diff x).
\end{align*}
- *strain and stress* tensors have their diagonal elements centered, and their off-diagonal elements shifted along the diagonal. For instance, the unknowns associated to the stress tensor are for $i,j \in \bZ$
\begin{align*}
    & \sigma_{00}((i,j)\diff x), &
    & \sigma_{11}((i,j)\diff x), &
    & \sigma_{01}((i+1/2,j+1/2)\diff x).
\end{align*}

### 2.1 Standard finite differences implementation

In [7]:
def eval_Virieux(qfun,pfun,X,dx,t,dt):
    """
    Evaluate position and momentum at the given position and time, 
    taking into account spatial and temporal grid shifts
    """
    vdim = len(X)
    t2 = t+dt/2
    if vdim==1:
        _,X_1 = shifted_grids(X,dx)
        q0_1 = qfun(t2,X_1)[0]
        p0_1 = pfun(t, X_1)[0]
        return (q0_1,),(p0_1,)
    if vdim==2:
        _,X_10,X_01,_ = shifted_grids(X,dx)
        q0_10 = qfun(t2,X_10)[0]
        q1_01 = qfun(t2,X_01)[1]
        p0_10 = pfun(t, X_10)[0]
        p1_01 = pfun(t, X_01)[1]
        return (q0_10,q1_01),(p0_10,p1_01)
    if vdim==3:
        X_000,X_100,X_010,X_110,X_001,X_101,X_011,X_111 = shifted_grids(X,dx)
        q0_100 = qfun(t2,X_100)[0]
        q1_010 = qfun(t2,X_010)[1]
        q2_001 = qfun(t2,X_001)[2]
        p0_100 = pfun(t, X_100)[0]
        p1_010 = pfun(t, X_010)[1]
        p2_001 = pfun(t, X_001)[2]
        return (q0_100,q1_010,q2_001),(p0_100,p1_010,p2_001)

In [8]:
class Virieux2:
    """The two dimensional Virieux scheme."""
    def __init__(self,ρ,C,stag):
        """
        Inputs : 
        - ρ : density, everywhere positive. 
        - C : Hooke tensor, assumes a VTI structure. Assumes C02=C12=0 and C symmetric.
        - stag : staggered grid difference operators
        """
        self.stag = stag
        ar = stag.avg_right
        # Variable locations on the grid shown after subscript
        iρ = 1/ρ
        self.iρ_10 = ar(iρ,0)
        self.iρ_01 = ar(iρ,1) 
        self.C00_00 = C[0,0]
        self.C01_00 = C[0,1]
        self.C11_00 = C[1,1] 
        self.C22_11 = ar(C[2,2],axis=(0,1))

    def step(self,q,p,dt):
        """
        Inputs : 
        - q,p : position and momentum.
        - dt : time step.
        """
        # Variable locations shown after subscript
        dl,dr = self.stag.diff_left, self.stag.diff_right
        C00_00,C01_00,C10_00,C11_00,C22_11,iρ_10,iρ_01 = self.C00_00,self.C01_00,self.C01_00,self.C11_00,self.C22_11,self.iρ_10,self.iρ_01

        q0_10,q1_01 = copy.deepcopy(q)
        p0_10,p1_01 = copy.deepcopy(p)
        
        # Compute the strain tensor ϵ = (Dq+Dq^T)/2
        ϵ00_00 = dl(q0_10,0) 
        ϵ11_00 = dl(q1_01,1) 
        ϵ01_11 = dr(q0_10,1) + dr(q1_01,0) # We omit the factor two in ϵ01 in view of Voigt's notation

        # Compute the stress tensor
        σ00_00 = C00_00*ϵ00_00+C01_00*ϵ11_00 
        σ11_00 = C10_00*ϵ00_00+C11_00*ϵ11_00 
        σ01_11 = C22_11*ϵ01_11 

        # Stress divergence
        dp0_10 = dr(σ00_00,0) + dl(σ01_11,1) 
        dp1_01 = dl(σ01_11,0) + dr(σ11_00,1)
#        self.tmp = (dp0_10,dp1_01),(σ00_00,σ11_00,σ01_11),(ϵ00_00,ϵ11_00,ϵ01_11)

        # Symplectic updates : first p, then q
        p0_10 += dt*dp0_10 
        p1_01 += dt*dp1_01 

        q0_10 += dt*p0_10*iρ_10 
        q1_01 += dt*p1_01*iρ_01

        return (q0_10,q1_01),(p0_10,p1_01)

In [9]:
class Virieux3:
    """The three dimensional Virieux scheme."""
    def __init__(self,ρ,C,stag):
        """
        Inputs : 
        - ρ : density, everywhere positive. 
        - C : Hooke tensor, assumes a VTI structure. Assumes C02=C12=0 and C symmetric.
        - stag : staggered grid difference operators
        """
        self.stag = stag
        ar = stag.avg_right
        # Variable locations on the grid shown after subscript
        iρ = 1/ρ
        self.iρ_100 = ar(iρ,0)
        self.iρ_010 = ar(iρ,1) 
        self.iρ_001 = ar(iρ,2) 
        self.C00_000 = C[0,0]
        self.C01_000 = C[0,1]
        self.C02_000 = C[0,2]
        self.C11_000 = C[1,1] 
        self.C12_000 = C[1,2] 
        self.C22_000 = C[2,2] 
        # Voigt : 3 -> (1,2), 4->(0,2), 5->(0,1)
        self.C33_011 = ar(C[3,3],axis=(1,2))
        self.C44_101 = ar(C[4,4],axis=(0,2))
        self.C55_110 = ar(C[5,5],axis=(0,1))

    def step(self,q,p,dt):
        """
        Inputs : 
        - q,p : position and momentum.
        - dt : time step.
        """
        # Variable locations shown after subscript
        dl,dr = self.stag.diff_left, self.stag.diff_right
        iρ_100,iρ_010,iρ_001, C00_000,C01_000,C02_000,C11_000,C12_000,C22_000, C33_011,C44_101,C55_110 = \
        self.iρ_100,self.iρ_010,self.iρ_001, self.C00_000,self.C01_000,self.C02_000,self.C11_000,self.C12_000,self.C22_000, self.C33_011,self.C44_101,self.C55_110

        q0_100,q1_010,q2_001 = copy.deepcopy(q)
        p0_100,p1_010,p2_001 = copy.deepcopy(p)
        
        # Compute the strain tensor ϵ = (Dq+Dq^T)/2
        ϵ00_000 = dl(q0_100,0) 
        ϵ11_000 = dl(q1_010,1)
        ϵ22_000 = dl(q2_001,2)
        ϵ01_110 = dr(q0_100,1) + dr(q1_010,0) # We omit the factor two in ϵ01 in view of Voigt's notation
        ϵ02_101 = dr(q0_100,2) + dr(q2_001,0) 
        ϵ12_011 = dr(q1_010,2) + dr(q2_001,1) 

        # Compute the stress tensor
        σ00_000 = C00_000*ϵ00_000 + C01_000*ϵ11_000 + C02_000*ϵ22_000
        σ11_000 = C01_000*ϵ00_000 + C11_000*ϵ11_000 + C12_000*ϵ22_000
        σ22_000 = C02_000*ϵ00_000 + C12_000*ϵ11_000 + C22_000*ϵ22_000
        # Voigt : 3 -> (1,2), 4->(0,2), 5->(0,1)
        σ12_011 = C33_011*ϵ12_011
        σ02_101 = C44_101*ϵ02_101
        σ01_110 = C55_110*ϵ01_110

        # Stress divergence
        dp0_100 = dr(σ00_000,0) + dl(σ01_110,1) + dl(σ02_101,2) 
        dp1_010 = dl(σ01_110,0) + dr(σ11_000,1) + dl(σ12_011,2) 
        dp2_001 = dl(σ02_101,0) + dl(σ12_011,1) + dr(σ22_000,2) 
        self.tmp = (dp0_100,dp1_010,dp2_001),(σ00_000,σ11_000,σ22_000,σ01_110,σ02_101,σ12_011),(ϵ00_000,ϵ11_000,ϵ22_000,ϵ01_110,ϵ02_101,ϵ12_011)

        # Symplectic updates : first p, then q
        p0_100 += dt*dp0_100 
        p1_010 += dt*dp1_010 
        p2_001 += dt*dp2_001 

        q0_100 += dt*p0_100*iρ_100 
        q1_010 += dt*p1_010*iρ_010
        q2_001 += dt*p2_001*iρ_001

        return (q0_100,q1_010,q2_001),(p0_100,p1_010,p2_001)

For completeness, we also implement the one dimensional case.

In [10]:
class Virieux1:
    """The two dimensional Virieux scheme."""
    def __init__(self,ρ,C,stag):
        """
        Inputs : 
        - ρ : density, everywhere positive. 
        - C : Hooke tensor, assumes a VTI structure. Assumes C02=C12=0 and C symmetric.
        - stag : staggered grid difference operators
        """
        self.stag = stag
        ar = stag.avg_right
        # Variable locations on the grid shown after subscript
        iρ = 1/ρ
        self.iρ_1 = ar(iρ,0)
        self.C00_0 = C[0,0]

    def step(self,q,p,dt):
        """
        Inputs : 
        - q,p : position and momentum.
        - dt : time step.
        """
        # Variable locations shown after subscript
        dl,dr = self.stag.diff_left, self.stag.diff_right
        C00_0,iρ_1 = self.C00_0,self.iρ_1

        q0_1, = copy.deepcopy(q)
        p0_1, = copy.deepcopy(p)
        
        # Compute the strain tensor ϵ = (Dq+Dq^T)/2
        ϵ00_0 = dl(q0_1,0) 

        # Compute the stress tensor
        σ00_0 = C00_0*ϵ00_0

        # Stress divergence
        dp0_1 = dr(σ00_0,0)

        # Symplectic updates : first p, then q
        p0_1 += dt*dp0_1 
        q0_1 += dt*p0_1*iρ_1 

        return (q0_1,),(p0_1,)

### 2.2 As a symplectic scheme

We implement the Virieux scheme as a symplectic scheme. On the positive side, we import all the associated techniques (higher order schemes, invariant energies, backpropagation, ...). One could also argue that this implementation is more compact and clear. On the negative side, there is the overhead of storing a sparse matrix, and of the initial step where this matrix is assembled. 

In [11]:
def VirieuxH1(ρ,C,stag,X):
    dl = stag.diff_left
    s = Virieux1(ρ,C,stag)
    def PotentialEnergy(q):
        q0_1, = q
        ϵ00_0 = dl(q0_1,0) # Compute the strain tensor ϵ = (Dq+Dq^T)/2
        return 0.5*s.C00_0*ϵ00_0**2
    def KineticEnergy(p):
        p0_1, = p
        return 0.5*s.iρ_1*p0_1**2
    H = QuadraticHamiltonian(PotentialEnergy,KineticEnergy)
    H.set_spmat(np.zeros_like(X)) # Replaces quadratic functions with sparse matrices
    return H

def VirieuxH2(ρ,C,stag,X,S=None):
    dl,dr = stag.diff_left,stag.diff_right
    s = Virieux2(ρ,C,stag)
    def PotentialEnergy(q):
        q0_10,q1_01 = q
        ϵ00_00 = dl(q0_10,0) # Compute the strain tensor ϵ = (Dq+Dq^T)/2
        ϵ11_00 = dl(q1_01,1) 
        ϵ01_11 = dr(q0_10,1) + dr(q1_01,0) # We omit the factor two in ϵ01 in view of Voigt's notation
        if S is not None: # Additional term arising in topographic changes of variables
            S00_00 = S[0,0]
            S11_00 = S[1,1]
            S01_11 = ar(2*S[0,1],(1+0,1+1))
            q_00 = ad.array((al(q0_10,0),al(q1_01,1)))
            q_11 = ad.array((ar(q0_10,1),ar(q1_01,1)))
            for ϵ_,S_,q_ in ((ϵ_00,S_00,q_00),(ϵ_11,S_11,q_11)): ϵ_ -= np.sum(S_*q_,axis=2)
        return 0.5*(s.C00_00*ϵ00_00**2 + 2*s.C01_00*ϵ00_00*ϵ11_00 + s.C11_00*ϵ11_00**2 + s.C22_11*ϵ01_11**2)
    def KineticEnergy(p):
        p0_10,p1_01 = p
        return 0.5*(s.iρ_10*p0_10**2 + s.iρ_01*p1_01**2)
    H = QuadraticHamiltonian(PotentialEnergy,KineticEnergy); H.set_spmat(np.zeros_like(X)); return H
    
def VirieuxH3(ρ,C,stag,X):
    dl,dr = stag.diff_left,stag.diff_right
    s = Virieux3(ρ,C,stag)
    def PotentialEnergy(q):
        q0_100,q1_010,q2_001 = q
        # Compute the strain tensor ϵ = (Dq+Dq^T)/2
        ϵ00_000 = dl(q0_100,0) 
        ϵ11_000 = dl(q1_010,1)
        ϵ22_000 = dl(q2_001,2)
        ϵ01_110 = dr(q0_100,1) + dr(q1_010,0) # We omit the factor two in ϵ01 in view of Voigt's notation
        ϵ02_101 = dr(q0_100,2) + dr(q2_001,0) 
        ϵ12_011 = dr(q1_010,2) + dr(q2_001,1) 
        # Compute the stress tensor
        σ00_000 = s.C00_000*ϵ00_000 + s.C01_000*ϵ11_000 + s.C02_000*ϵ22_000
        σ11_000 = s.C01_000*ϵ00_000 + s.C11_000*ϵ11_000 + s.C12_000*ϵ22_000
        σ22_000 = s.C02_000*ϵ00_000 + s.C12_000*ϵ11_000 + s.C22_000*ϵ22_000
        # Voigt : 3 -> (1,2), 4->(0,2), 5->(0,1)
        σ12_011 = s.C33_011*ϵ12_011
        σ02_101 = s.C44_101*ϵ02_101
        σ01_110 = s.C55_110*ϵ01_110
        return 0.5*(ϵ00_000*σ00_000 + ϵ11_000*σ11_000 + ϵ22_000*σ22_000 + ϵ01_110*σ01_110 + ϵ02_101*σ02_101 + ϵ12_011*σ12_011)
    def KineticEnergy(p):
        p0_100,p1_010,p2_001 = p
        return 0.5*(s.iρ_100*p0_100**2 + s.iρ_010*p1_010**2 + s.iρ_001*p2_001**2)
    H = QuadraticHamiltonian(PotentialEnergy,KineticEnergy); H.set_spmat(np.zeros_like(X)); return H

Let us check that the different scheme implementations yield identical results.

In [12]:
for vdim,Nx,order_x in [
    (1,10,2),(2,12,2),(2,12,4),(3,9,2)
]:
    np.random.seed(42)
    shape = (Nx,)*vdim
    X,dx = make_domain(Nx,vdim)
    stag = staggered(dx,order_x)
    q0 = np.random.rand(vdim,*shape)
    p0 = np.random.rand(vdim,*shape)
    ρ = 0.5+np.random.rand(*shape)
    symdim = int((vdim*(vdim+1))//2)
    C = MakeRandomTensor(symdim,shape,relax=0.5) 
    # Set the non-VTI coefficients to zero (would be ignored anyway)
    for i in range(symdim): 
        for j in range(symdim):
            if i>=vdim and j>=vdim and i!=j: C[i,j]=0
    dt = 0.1
    
    Virieux  = (None,Virieux1, Virieux2, Virieux3 )[vdim]    
    VirieuxH = (None,VirieuxH1,VirieuxH2,VirieuxH3)[vdim]  
    scheme  = Virieux( ρ,C,stag)
    schemeH = VirieuxH(ρ,C,stag,X)
    q1,p1   = scheme.step(    q0,p0,dt)
    q1H,p1H = schemeH.Euler_p(q0,p0,dt)

    assert np.allclose(q1,q1H) 
    assert np.allclose(p1,p1H) 

## 3. The Lebedev scheme

The two dimensional Lebedev scheme consists of two Virieux schemes, weakly coupled through the coefficients $C_{02}$ and $C_{12}$ of the Hooke tensor. There are four coupled Virieux schemes in three dimensions. On the other hand, one dimensional Lebedev scheme is equivalent to the Virieux scheme.

The Lebedev scheme uses a staggered grid, and locates the variables as follows.  Compare with the Virieux schemes which stores only one component at each of these positions. 

- *position and momentum*. All components are stored at $(i+1/2,j)\diff x$ and $(i,j+1/2)\diff x$ in two dimensions. (At $(i+1/2,j,k)\diff x$, $(i,j+1/2,k)\diff x$, $(i,j,k+1/2)\diff x$ and $(i+1/2,j+1/2,k+1/2)\diff x$ in three dimensions.)
- *stress and strain*. All components are stored at $(i,j)\diff x$ and $(i+1/2,j+1/2) \diff x$ in two dimensions. (At $(i,j,k)\diff x$, $(i+1/2,j+1/2,k)\diff x$, $(i+1/2,j,k+1/2)\diff x$ and $(i,j+1/2,k+1/2)\diff x$ in three dimensions.)

### 3.1 Finite differences implementation

In [13]:
def eval_Lebedev(qfun,pfun,X,dx,t,dt):
    """
    Evaluate position and momentum at the given position and time, 
    taking into account spatial and temporal grid shifts
    """
    t2=t+dt/2
    vdim=len(X)
    if vdim==1:
        _,X_1 = shifted_grids(X,dx)
        return qfun(t2,X_1),pfun(t,X_1)
    if vdim==2:
        _,X_10,X_01,_ = shifted_grids(X,dx)
        q_10 = qfun(t2,X_10)
        q_01 = qfun(t2,X_01)
        p_10 = pfun(t ,X_10)
        p_01 = pfun(t ,X_01)
        return (q_10,q_01),(p_10,p_01) # Geometry is second
    if vdim==3:
        X_000,X_100,X_010,X_110,X_001,X_101,X_011,X_111 = shifted_grids(X,dx)
        q_100 = qfun(t2,X_100)
        q_010 = qfun(t2,X_010)
        q_001 = qfun(t2,X_001)
        q_111 = qfun(t2,X_111)
        p_100 = pfun(t ,X_100)
        p_010 = pfun(t ,X_010)
        p_001 = pfun(t ,X_001)
        p_111 = pfun(t ,X_111)
        return (q_100,q_010,q_001,q_111), (p_100,p_010,p_001,p_111)

In [14]:
class Lebedev2:
    """The two dimensional Lebedev scheme"""
    def __init__(self,ρ,C,stag):
        """
        Inputs : 
        - ρ : density, everywhere positive. 
        - C : Hooke tensor, assumed to be symmetric.
        - stag : staggered grid difference operators
        """
        self.stag = stag
        ar = stag.avg_right
        # Variable location on the grid shown after underscore
        iρ = 1/ρ 
        self.iρ_10 = ar(iρ,0) 
        self.iρ_01 = ar(iρ,1) 
        self.C_00 = C 
        self.C_11 = ar(C,axis=(2+0,2+1)) 

    def step(self,q,p,dt):
        """
        Inputs : 
        - q,p : position and momentum.
        - dt : time step.
        """
        dl,dr = self.stag.diff_left, self.stag.diff_right
        C_00,C_11,iρ_10,iρ_01 = self.C_00,self.C_11,self.iρ_10,self.iρ_01
         
        # Variable location on the grid shown after underscore
        (q0_10,q1_10), (q0_01,q1_01) = copy.deepcopy(q)
        (p0_10,p1_10), (p0_01,p1_01) = copy.deepcopy(p)

        # Compute the strain tensor ϵ = (Dq+Dq^T)/2
        ϵ00_00 = dl(q0_10,0) 
        ϵ11_00 = dl(q1_01,1) 
        ϵ01_00 = dl(q0_01,1)+dl(q1_10,0) 

        ϵ00_11 = dr(q0_01,0)
        ϵ11_11 = dr(q1_10,1)
        ϵ01_11 = dr(q0_10,1) + dr(q1_01,0) # We omit the factor two in ϵ01 in view of Voigt's notation
        
        # Compute the stress tensor
        σ00_00 = C_00[0,0]*ϵ00_00 + C_00[0,1]*ϵ11_00 + C_00[0,2]*ϵ01_00 
        σ11_00 = C_00[1,0]*ϵ00_00 + C_00[1,1]*ϵ11_00 + C_00[1,2]*ϵ01_00 
        σ01_00 = C_00[2,0]*ϵ00_00 + C_00[2,1]*ϵ11_00 + C_00[2,2]*ϵ01_00

        σ00_11 = C_11[0,0]*ϵ00_11 + C_11[0,1]*ϵ11_11 + C_11[0,2]*ϵ01_11 
        σ11_11 = C_11[1,0]*ϵ00_11 + C_11[1,1]*ϵ11_11 + C_11[1,2]*ϵ01_11 
        σ01_11 = C_11[2,0]*ϵ00_11 + C_11[2,1]*ϵ11_11 + C_11[2,2]*ϵ01_11

        # Stress divergence
        dp0_10 = dr(σ00_00,0) + dl(σ01_11,1) 
        dp1_01 = dl(σ01_11,0) + dr(σ11_00,1) 

        dp0_01 = dl(σ00_11,0) + dr(σ01_00,1) 
        dp1_10 = dr(σ01_00,0) + dl(σ11_11,1) 
#        self.tmp = (dp0_10,dp1_01),(σ00_00,σ11_00,σ01_11),(ϵ00_00,ϵ11_00,ϵ01_11)

        # Symplectic updates : first p, then q
        p0_10 += dt*dp0_10 
        p1_10 += dt*dp1_10 
        p0_01 += dt*dp0_01 
        p1_01 += dt*dp1_01 

        q0_10 += dt*p0_10*iρ_10
        q1_10 += dt*p1_10*iρ_10
        q0_01 += dt*p0_01*iρ_01
        q1_01 += dt*p1_01*iρ_01

        return ((q0_10,q1_10), (q0_01,q1_01)), ((p0_10,p1_10), (p0_01,p1_01))

In [15]:
class Lebedev3:
    """The three dimensional Lebedev scheme"""
    def __init__(self,ρ,C,stag):
        self.stag = stag
        ar = stag.avg_right
        # Variable location on the grid shown after underscore
        iρ = 1/ρ 
        self.iρ_100 = ar(iρ,0) 
        self.iρ_010 = ar(iρ,1)
        self.iρ_001 = ar(iρ,2)
        self.iρ_111 = ar(iρ,(0,1,2))
        self.C_000 = C 
        self.C_110 = ar(C,axis=(2+0,2+1)) 
        self.C_101 = ar(C,axis=(2+0,2+2)) 
        self.C_011 = ar(C,axis=(2+1,2+2)) 

    def step(self,q,p,dt):
        q_100,q_010,q_001,q_111 = copy.deepcopy(q) 
        p_100,p_010,p_001,p_111 = copy.deepcopy(p)
        dl,dr = self.stag.diff_left, self.stag.diff_right
        iρ_100,iρ_010,iρ_001,iρ_111, C_000,C_110,C_101,C_011 = self.iρ_100,self.iρ_010,self.iρ_001,self.iρ_111, self.C_000,self.C_110,self.C_101,self.C_011

        # Compute the strain tensor ϵ = (Dq+Dq^T)/2
        # Voigt convention : 00,11,22, 12,02,01
        ϵ_000 = (dl(q_100[0],0), dl(q_010[1],1), dl(q_001[2],2), dl(q_001[1],2)+dl(q_010[2],1), dl(q_001[0],2)+dl(q_100[2],0), dl(q_010[0],1)+dl(q_100[1],0) )
        ϵ_110 = (dr(q_010[0],0), dr(q_100[1],1), dl(q_111[2],2), dl(q_111[1],2)+dr(q_100[2],1), dl(q_111[0],2)+dr(q_010[2],0), dr(q_100[0],1)+dr(q_010[1],0) )
        ϵ_101 = (dr(q_001[0],0), dl(q_111[1],1), dr(q_100[2],2), dr(q_100[1],2)+dl(q_111[2],1), dr(q_100[0],2)+dr(q_001[2],0), dl(q_111[0],1)+dr(q_001[1],0) )
        ϵ_011 = (dl(q_111[0],0), dr(q_001[1],1), dr(q_010[2],2), dr(q_010[1],2)+dr(q_001[2],1), dr(q_010[0],2)+dl(q_111[2],0), dr(q_001[0],1)+dl(q_111[1],0) )

        # Compute the stress tensor, as a matrix-vector product
        σ_000 = [sum(C_000[i,j]*ϵ_000[j] for j in range(6)) for i in range(6)]
        σ_110 = [sum(C_110[i,j]*ϵ_110[j] for j in range(6)) for i in range(6)]
        σ_101 = [sum(C_101[i,j]*ϵ_101[j] for j in range(6)) for i in range(6)]
        σ_011 = [sum(C_011[i,j]*ϵ_011[j] for j in range(6)) for i in range(6)]

        # Stress divergence  [0,5,4]
        # Voigt convention   [5,1,3]
        #                    [4,3,2]
        dp_100 = (dr(σ_000[0],0) + dl(σ_110[5],1) + dl(σ_101[4],2), 
                  dr(σ_000[5],0) + dl(σ_110[1],1) + dl(σ_101[3],2), 
                  dr(σ_000[4],0) + dl(σ_110[3],1) + dl(σ_101[2],2) )
        
        dp_010 = (dl(σ_110[0],0) + dr(σ_000[5],1) + dl(σ_011[4],2), 
                  dl(σ_110[5],0) + dr(σ_000[1],1) + dl(σ_011[3],2), 
                  dl(σ_110[4],0) + dr(σ_000[3],1) + dl(σ_011[2],2) )
        
        dp_001 = (dl(σ_101[0],0) + dl(σ_011[5],1) + dr(σ_000[4],2), 
                  dl(σ_101[5],0) + dl(σ_011[1],1) + dr(σ_000[3],2), 
                  dl(σ_101[4],0) + dl(σ_011[3],1) + dr(σ_000[2],2) )
        
        dp_111 = (dr(σ_011[0],0) + dr(σ_101[5],1) + dr(σ_110[4],2), 
                  dr(σ_011[5],0) + dr(σ_101[1],1) + dr(σ_110[3],2), 
                  dr(σ_011[4],0) + dr(σ_101[3],1) + dr(σ_110[2],2) )

        self.tmp = (dp_100,dp_010,dp_001,dp_111), (σ_000,σ_110,σ_101,σ_011), (ϵ_000,ϵ_110,ϵ_101,ϵ_011)
        
        # Symplectic updates : first p, then q
        for i in range(3):
            p_100[i] += dt*dp_100[i]
            p_010[i] += dt*dp_010[i]
            p_001[i] += dt*dp_001[i]
            p_111[i] += dt*dp_111[i]

        for i in range(3):
            q_100[i] += dt*iρ_100*p_100[i]
            q_010[i] += dt*iρ_010*p_010[i]
            q_001[i] += dt*iρ_001*p_001[i]
            q_111[i] += dt*iρ_111*p_111[i]

        return (q_100,q_010,q_001,q_111), (p_100,p_010,p_001,p_111)

### 3.2 Symplectic implementation

We provide a symplectic implementation of the Lebedev scheme, with the same tradeoffs as for the Virieux case.

In [16]:
def LebedevH2(ρ,C,stag,X):
    dl,dr,al,ar = stag.diff_left,stag.diff_right,stag.avg_left,stag.avg_right
    s = Lebedev2(ρ,C,stag)
    def PotentialEnergy(q):
        q_10,q_01 = q
        # Compute the strain tensor ϵ = (Dq+Dq^T)/2. Voigt convention : 00,11, 01
        ϵ_00 = ad.array((dl(q_10[0],0),dl(q_01[1],1), dl(q_01[0],1)+dl(q_10[1],0)))
        ϵ_11 = ad.array((dr(q_01[0],0),dr(q_10[1],1), dr(q_10[0],1)+dr(q_01[1],0)))
        return 0.5*sum(lp.dot_VAV(ϵ_,C_,ϵ_) for (ϵ_,C_) in ((ϵ_00,s.C_00),(ϵ_11,s.C_11)))
    def KineticEnergy(p):
        p_10,p_01 = p
        return 0.5*sum(p_**2 * iρ_ for (p_,iρ_) in ((p_10,s.iρ_10),(p_01,s.iρ_01)))
    H = QuadraticHamiltonian(PotentialEnergy,KineticEnergy); H.set_spmat(np.zeros_like(X,shape=(2,*X.shape))); return H
     
def LebedevH3(ρ,C,stag,X):
    dl,dr = stag.diff_left,stag.diff_right
    s = Lebedev3(ρ,C,stag)
    def PotentialEnergy(q):
        q_100,q_010,q_001,q_111 = q
        # Compute the strain tensor ϵ = (Dq+Dq^T)/2. Voigt convention : 00,11,22, 12,02,01
        ϵ_000 = ad.array((dl(q_100[0],0), dl(q_010[1],1), dl(q_001[2],2), dl(q_001[1],2)+dl(q_010[2],1), dl(q_001[0],2)+dl(q_100[2],0), dl(q_010[0],1)+dl(q_100[1],0) ))
        ϵ_110 = ad.array((dr(q_010[0],0), dr(q_100[1],1), dl(q_111[2],2), dl(q_111[1],2)+dr(q_100[2],1), dl(q_111[0],2)+dr(q_010[2],0), dr(q_100[0],1)+dr(q_010[1],0) ))
        ϵ_101 = ad.array((dr(q_001[0],0), dl(q_111[1],1), dr(q_100[2],2), dr(q_100[1],2)+dl(q_111[2],1), dr(q_100[0],2)+dr(q_001[2],0), dl(q_111[0],1)+dr(q_001[1],0) ))
        ϵ_011 = ad.array((dl(q_111[0],0), dr(q_001[1],1), dr(q_010[2],2), dr(q_010[1],2)+dr(q_001[2],1), dr(q_010[0],2)+dl(q_111[2],0), dr(q_001[0],1)+dl(q_111[1],0) ))
        return 0.5*sum(lp.dot_VAV(ϵ_,C_,ϵ_) for (ϵ_,C_) in ((ϵ_000,s.C_000),(ϵ_110,s.C_110),(ϵ_101,s.C_101),(ϵ_011,s.C_011)))
    def KineticEnergy(p):
        p_100,p_010,p_001,p_111 = p
        return 0.5*sum(p_**2 * iρ_ for (p_,iρ_) in ((p_100,s.iρ_100),(p_010,s.iρ_010),(p_001,s.iρ_001),(p_111,s.iρ_111)))
    H = QuadraticHamiltonian(PotentialEnergy,KineticEnergy); H.set_spmat(np.zeros_like(X,shape=(4,*X.shape))); return H
    

Let us check that the symplectic and original implementations yield identical results.

In [17]:
for vdim,Nx,order_x in [
    (2,12,2),(2,12,4),(3,9,2)
]:
    if vdim!=2: break
    np.random.seed(42)
    shape = (Nx,)*vdim
    X,dx = make_domain(Nx,vdim)
    stag = staggered(dx,order_x)
    q0 = np.random.rand(2**(vdim-1),vdim,*shape)
    p0 = np.random.rand(2**(vdim-1),vdim,*shape)
    ρ = 0.5+np.random.rand(*shape)
    symdim = int((vdim*(vdim+1))//2)
    C = MakeRandomTensor(symdim,shape,relax=0.5) 
    dt = 0.1
    
    Lebedev  = (None,None,Lebedev2, Lebedev3 )[vdim]    
    LebedevH = (None,None,LebedevH2,LebedevH3)[vdim]  
    scheme  = Lebedev( ρ,C,stag)
    schemeH = LebedevH(ρ,C,stag,X)
    q1,p1   = scheme.step(    q0,p0,dt)
    q1H,p1H = schemeH.Euler_p(q0,p0,dt)

    assert np.allclose(q1,q1H) 
    assert np.allclose(p1,p1H) 

In [109]:
def LebedevH2_ext(M,C,stag,X,S=None):
    dl,dr,al,ar = stag.diff_left,stag.diff_right,stag.avg_left,stag.avg_right
    M_10 = ar(M,2+0) 
    M_01 = ar(M,2+1) 
    C_00 = C 
    C_11 = ar(C,axis=(2+0,2+1)) 
    def PotentialEnergy(q):
        q_10,q_01 = q
        # Compute the strain tensor ϵ = (Dq+Dq^T)/2. Voigt convention : 00,11, 01
        ϵ_00 = ad.array((dl(q_10[0],0),dl(q_01[1],1), dl(q_01[0],1)+dl(q_10[1],0)))
        ϵ_11 = ad.array((dr(q_01[0],0),dr(q_10[1],1), dr(q_10[0],1)+dr(q_01[1],0)))
        if S is not None: # Additional term arising in topographic changes of variables
            S_00 = ad.array((S[0,0],S[1,1],2*S[0,1]))
            S_11 = ar(S_00,(2+0,2+1))
            q_00 = 0.5*(al(q_10,1+0)+al(q_01,1+1))
            q_11 = 0.5*(ar(q_10,1+1)+ar(q_01,1+0))
            for ϵ_,S_,q_ in ((ϵ_00,S_00,q_00),(ϵ_11,S_11,q_11)): ϵ_ -= np.sum(S_*q_,axis=1)
        return 0.5*sum(lp.dot_VAV(ϵ_,C_,ϵ_) for (ϵ_,C_) in ((ϵ_00,C_00),(ϵ_11,C_11)))
    def KineticEnergy(p):
        p_10,p_01 = p
        return 0.5*(p_10[None,:]*p_10[:,None]*M_10 + p_01[None,:]*p_01[:,None]*M_01) 
    H = QuadraticHamiltonian(PotentialEnergy,KineticEnergy); H.set_spmat(np.zeros_like(X,shape=(2,*X.shape))); return H

This slightly more general variant also works with the coefficients obtained through changes of variables, see the notebook on [high order schemes for the wave equations](HighOrderWaves.ipynb).

## 4. Dispersion relations

**Continuous setting.**
Consider a plane wave 
\begin{align*}
    q(t,x) &= w \be(\<\kw,x\> - \omega t), &
    p(t,x) &= v \be(\<\kw,x\> - \omega t),
\end{align*}
where $\kw$ is the wavenumber, $\omega$ is the pulsation, and $\be(s) := \exp(\ri s)$.

If this wave obeys the elastic wave equation, with constant coefficients, then one easily checks that 
\begin{align*}
    -\ri\omega\, v &= C[\ri\kw] w, &
    -\ri\omega\, w &= v/\rho,
\end{align*}
where $C[\kw]_{il} := \sum_{jk} C_{ijkl} \kw_i \kw_l$.

Note that this relation implies that $w$ is an eigenvector of $C[\kw]$, more precisely $\rho \omega^2 w = C[\ri\kw] w$.

**Discrete setting.**
We first define two helper functions, which correspond to fourier transforms of first order finite difference operators:
\begin{align*}
    \rI_2^h(s) &
    := \frac{\exp(\ri s h/2) - \exp(\ri s h/2)} {\ri h} 
    = \frac{\sin(s h/2)}{h/2} 
    = s - \frac{ s^3 h^2} {24} +  \cO(h^4), \\
    \rI_4^h(s) &
    := \frac 9 8 \frac{\exp(\ri s h/2) - \exp(\ri s h/2)} {\ri h} -\frac 1 {24} \frac{\exp(3\ri s h/2) - \exp(3\ri s h/2)} {\ri h} 
    = \frac 9 8 \frac{\sin(s h/2)}{h/2}  - \frac 1 {24} \frac{\sin(3 s h/2)}{h/2}
    = s - \frac {3 s^5 h^4}{640} + \cO(h^6).
\end{align*}
These expression are second order and fourth order approximations of the identity, respectively. The original ``staggered'' one dimensional finite difference operators are 
\begin{align*}
    \frac{u(x+h/2)-u(x-h/2)} h &= u'(x) + \cO(h^2), \\
    \frac 9 8 \frac{u(x+h/2)-u(x-h/2)} h - \frac 1 8 \frac{u(x+3h/2)-u(x-3h/2)} {3h} &= u'(x) + \cO(h^4).
\end{align*}


In view of the finite difference expressions, and of the locations and times where $q$ and $p$ are evaluated, one obtains the discrete dispersion relation
\begin{align*}
    -\ri \rI_2^{\diff t} (\omega)\, v &= C[\ri \rI_{ox}^{\diff x}\kw] w, &
    -\ri \rI_2^{\diff t}(\omega)\, w &= v/\rho,
\end{align*}
where $ox$ denotes the order of the scheme w.r.t the space variable. This relation implies $\rho \rI_2^{\diff t}(\omega)^2 w = C[\rI_{ox}^{\diff x}\kw] w$.
Given a wave number $\kw$, one can diagonalize the symmetric matrix $C[\rI_{ox}^{\diff x}\kw]$, and thus find the amplitude and pulsations of the corresponding propagation modes.

In [18]:
def _dispersion_DiffStaggered(s,h,order=2):
    """
    Approximation of identity, corresponding to the Fourier transform 
    of first order finite difference operators on staggered grids. Denoted I_order^h in text.
    """
    h2 = h/2; sh2 = s*h/2
    if order==2: return np.sin(sh2)/h2
    if order==4: return (9/8)*np.sin(sh2)/h2-(1/24)*np.sin(3*sh2)/h2
    raise ValueError("Unsupported order")

def _dispersion_AvgStaggered(s,h,order=2):
    """
    Fourier transform of the midpoint interpolation operator on staggered grid.
    """
    sh2 = s*h/2
    if order==2: return np.cos(sh2)
    if order==4: return (9/8)*np.cos(sh2)-(1/8)*np.cos(3*sh2)
    raise ValueError("Unsupported order")    

def _dispersion_Iinv(s,h):
    """Inverse of I(s,h,order=2), where I is _dispersion_DiffStaggered"""
    h2=h/2
    return np.arcsin(s*h2)/h2

def _dispersion_ElasticT2(ρ,Ck,dt):
    """Dispersion of a second order time discretization of an elastic wave equation"""
    if np.ndim(Ck)==1: Iω2,vq = np.linalg.eigh(Ck/ρ)
    else: Iω2,vq = np.linalg.eigh(np.moveaxis(Ck/ρ,(0,1),(-2,-1))); Iω2 = np.moveaxis(Iω2,-1,0); vq = np.moveaxis(vq,(-2,-1),(0,1)) 
    
    Iω = np.sqrt(Iω2)
    ω = _dispersion_Iinv(Iω,dt)
    vp = -ρ*Iω*vq # Omitting multiplication by i
    return ω,vq,vp

def dispersion_Virieux(k,ρ,C,dx,dt,order_x=2):
    """
    Dispersion relation for the Virieux and Lebedev schemes (but Virieux expects C02=C12=0)
    returns the wavemodes and pulsations corresponding to the wavenumber k.
    """
    Ik = _dispersion_DiffStaggered(k,dx,order_x)
    Ck = Hooke(C).contract(Ik)
    return _dispersion_ElasticT2(ρ,Ck,dt)

def mk_planewave_e(k,ω,vq,vp):
    """Make an elastic planewave with the specified parameters."""
    def brdcst(v,x): return fd.as_field(v,x[0].shape,conditional=False)
    def expi(t,x): return np.exp(1j*(lp.dot_VV(brdcst(k,x),x) - ω*t)) 
    def q_exact(t,x): return    expi(t,x) * brdcst(vq,x)
    def p_exact(t,x): return 1j*expi(t,x) * brdcst(vp,x)
    return q_exact,p_exact

Let us cross-check our implementations and these dispersion relations, beginning with two-dimensional schemes.

In [19]:
C_VTI2 = Hooke.mica[0].extract_xz().hooke
C_TTI2 = Hooke.mica[0].extract_xz().rotate_by(π/6).hooke
C_VTI3 = Hooke.mica[0].hooke
C_TTI3 = Hooke.mica[0].rotate_by(π/6,(1,2,3)).hooke
for C in (C_VTI2,C_TTI2,C_VTI3,C_TTI3): C/=np.max(C)

for ρ, C_VTI, C, k, order_x, Nx, dt, mode in [
    (1.4, C_VTI2, C_TTI2, [ 2, 3],    2, 10, 0.01, 0),
    (0.9, C_VTI2, C_TTI2, [-1, 1],    4, 10, 0.01, 1),
    (1.6, C_VTI3, C_TTI3, [ 2, 1, 3], 2, 10, 0.01, 1),
    (1.9, C_VTI3, C_TTI3, [-1, 1, 2], 4, 10, 0.01, 0),
]:
    vdim = len(k)
    k = 2*π*np.array(k)
    X, dx = make_domain(Nx,vdim) # The grid points : offset (0,0)
    stag = staggered(dx,order_x,'Periodic')

    # Time discretization
    Nt = 1
    T = Nt*dt

    # Dispersion relation and exact solution
    ω,vq,vp = dispersion_Virieux(k,ρ,C_VTI,dx,dt,order_x)
    q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])

    q0,p0 = eval_Virieux(q_exact,p_exact,X,dx,0,dt)
    qf,pf = eval_Virieux(q_exact,p_exact,X,dx,T,dt)

    def af(q): return fd.as_field(q,shape=X[0].shape)
    Virieux = (None,Virieux1,Virieux2,Virieux3)[vdim] # Select adequate scheme
    scheme = Virieux(af(ρ),af(C_VTI),stag)
    q1,p1 = scheme.step(q0,p0,dt)
    assert np.allclose(qf,q1)
    assert np.allclose(pf,p1)

    # Now testing the Lebedev scheme
    if vdim==1: continue
    Q0,P0 = eval_Lebedev(q_exact,p_exact,X,dx,0,dt)
    Qf,Pf = eval_Lebedev(q_exact,p_exact,X,dx,T,dt)
    Lebedev = (None,None,Lebedev2,Lebedev3)[vdim]
    Scheme = Lebedev(af(ρ),af(C_VTI),stag)
    Q1,P1 = Scheme.step(Q0,P0,dt)
    assert np.allclose(Qf,Q1)
    assert np.allclose(Pf,P1)

    # For a VTI model, the Lebedev scheme incorporates several decoupled Virieux schemes
    for i in range(vdim): assert np.allclose(Qf[i][i],qf[i])

    # However, the Lebedev scheme also handles general elasticity
    ω,vq,vp = dispersion_Virieux(k,ρ,C,dx,dt,order_x)
    q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])
    Q0,P0 = eval_Lebedev(q_exact,p_exact,X,dx,0,dt)
    Qf,Pf = eval_Lebedev(q_exact,p_exact,X,dx,T,dt)
    Scheme = Lebedev(af(ρ),af(C),stag)
    Q1,P1 = Scheme.step(Q0,P0,dt)
    assert np.allclose(Qf,Q1)
    assert np.allclose(Pf,P1)


(Old test code commented.)

<!---
# Virieux
ω=ω[mode]; vq=vq[:,mode]; vp=vp[:,mode]
(dp0_10,dp1_01),(σ00_00,σ11_00,σ01_11),(ϵ00_00,ϵ11_00,ϵ01_11) = scheme.tmp

def expi(x): return np.exp(1j*(k[0]*x[0]+k[1]*x[1]-ω*dt/2))
# This works for any vq, not necessarily an eigenvector
assert np.allclose(q0[0]/expi(X_10),vq[0])
assert np.allclose(ϵ00_00/expi(X_00),vq[0]*dspi(0))
assert np.allclose(ϵ11_00/expi(X_00),vq[1]*dspi(1))
assert np.allclose(ϵ01_11/expi(X_11),vq[0]*dspi(1)+vq[1]*dspi(0))
assert np.allclose(σ00_00/expi(X_00),C[0,0]*vq[0]*dspi(0) + C[0,1]*vq[1]*dspi(1))
assert np.allclose(σ11_00/expi(X_00),C[1,0]*vq[0]*dspi(0) + C[1,1]*vq[1]*dspi(1))
assert np.allclose(σ01_11/expi(X_11),C[2,2]*(vq[0]*dspi(1)+vq[1]*dspi(0)))
assert np.allclose(dp0_10/expi(X_10),(C[0,0]*vq[0]*dspi(0)+C[0,1]*vq[1]*dspi(1))*dspi(0) + C[2,2]*(vq[0]*dspi(1)+vq[1]*dspi(0))*dspi(1))
assert np.allclose(dp1_01/expi(X_01),C[2,2]*(vq[0]*dspi(1)+vq[1]*dspi(0))*dspi(0)+(C[1,0]*vq[0]*dspi(0) + C[1,1]*vq[1]*dspi(1))*dspi(1))
Ik = _dispersion_I(k,dx,order_x)
Iω = _dispersion_I(ω,dt)
Ck = Hooke(C).contract(Ik)
assert np.allclose(dp0_10/expi(X_10),-(Ck@vq)[0])
assert np.allclose(dp1_01/expi(X_01),-(Ck@vq)[1])
# Below, need chosen vq
assert np.allclose(Iω*vq,-vp/ρ)
assert np.allclose(Iω*vp,-Ck@vq)
assert np.allclose(p1,pf)

#Iω*vp,Ck@vq, ρ*Iω*vq,vp
#assert np.allclose(_dispersion_I(ω,dt)*vp,Ck@vq)
#Ck@vq,(C[0,0]*vq[0]*dspi(0) + C[0,1]*vq[1]*dspi(1))*dspi(0) + C[2,2]*(vq[0]*dspi(1)+vq[1]*dspi(0))*dspi(1),C[2,2]*(vq[0]*dspi(1)+vq[1]*dspi(0))*dspi(0)+(C[1,0]*vq[0]*dspi(0) + C[1,1]*vq[1]*dspi(1))*dspi(1)

#np.array(q1)-np.array(qf)
def expi(x): return np.exp(1j*(k[0]*x[0]+k[1]*x[1]-ω*dt))
assert np.allclose((qf[0]-q0[0])/expi(X_10),-1j*vq[0]*Iω*dt)
assert np.allclose(pf[0]/expi(X_10),1j*vp[0])
assert np.allclose(qf[0]-q0[0], pf[0]*dt/ρ)
assert np.allclose(q1[0]-q0[0], pf[0]*dt/ρ)

# Lebedev
assert np.allclose(Q0[0][0],q0[0])
assert np.allclose(Q0[1][1],q0[1])
assert np.allclose(P0[0][0],p0[0])
assert np.allclose(P0[1][1],p0[1])

assert np.allclose(Qf[0][0],qf[0])
assert np.allclose(Q1[0][0],q1[0])
assert np.allclose(Qf[0][0],Q1[0][0])
--->

<!---
#C = Hooke.from_Lame(0,1,vdim=2).hooke; ρ=1
C = Hooke.mica[0].extract_xz().hooke; C/=np.max(C); ρ=1.5
k = 2*π*np.array([2,3])
order_x = 2

# The domain is the periodic unit square
Nx = 10
bc = 'Periodic'
X, dx = make_domain(Nx,2) # The grid points : offset (0,0)
stag = staggered(dx,order_x,bc)

# Check that _dispersion_I is indeed the Fourier transform of the finite differences operator
def expi(x): return np.exp(1j*(k[0]*x[0]+k[1]*x[1]))
def dspi(axis): return 1j*_dispersion_I(k[axis],dx,order_x)
X_00,X_10,X_01,X_11 = shifted_grids(X,dx)
assert np.allclose(stag.diff_left( expi(X_10),0),     expi(X)*dspi(0))
assert np.allclose(stag.diff_right(expi(2*X-X_10),0), expi(X)*dspi(0))
assert np.allclose(stag.diff_left( expi(X_01),1),     expi(X)*dspi(1))
assert np.allclose(stag.diff_right(expi(2*X-X_01),1), expi(X)*dspi(1))

# Time step
dt = 0.01
Nt = 1
T = Nt*dt

# Dispersion relation and exact solution
ω,vq,vp = dispersion_Virieux(k,ρ,C,dx,dt)
mode = 0
q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])

q0,p0 = eval_Virieux(q_exact,p_exact,X,dx,0,dt)
qf,pf = eval_Virieux(q_exact,p_exact,X,dx,T,dt)

def af(q): return fd.as_field(q,shape=X[0].shape)
scheme = Virieux2(af(ρ),af(C),stag)
q1,p1 = scheme.step(q0,p0,dt)
assert np.allclose(qf,q1)
assert np.allclose(pf,p1)

# Now testing the Lebedev scheme
Q0,P0 = eval_Lebedev(q_exact,p_exact,X,dx,0,dt)
Qf,Pf = eval_Lebedev(q_exact,p_exact,X,dx,T,dt)
Scheme = Lebedev2(af(ρ),af(C),stag)
Q1,P1 = Scheme.step(Q0,P0,dt)
assert np.allclose(Qf,Q1)
assert np.allclose(Pf,P1)

# For a VTI model, the Lebedev scheme consists of two decoupled Virieux schemes
assert np.allclose(Qf[0][0],qf[0])
assert np.allclose(Qf[1][1],qf[1])

# However, the Lebedev scheme also handles general elasticity
C = Hooke.mica[0].extract_xz().rotate_by(π/6).hooke; C/=np.max(C); ρ=1.5
ω,vq,vp = dispersion_Virieux(k,ρ,C,dx,dt)
mode = 1
q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])
Q0,P0 = eval_Lebedev(q_exact,p_exact,X,dx,0,dt)
Qf,Pf = eval_Lebedev(q_exact,p_exact,X,dx,T,dt)
Scheme = Lebedev2(af(ρ),af(C),stag)
Q1,P1 = Scheme.step(Q0,P0,dt)
assert np.allclose(Qf,Q1)
assert np.allclose(Pf,P1)
--->

<!---
C = Hooke.mica[0].hooke; C/=np.max(C); ρ=1.5
k = 2*π*np.array([2,-1,3])
order_x = 2

# The domain is the periodic unit square
Nx = 10
bc = 'Periodic'
X, dx = make_domain(10,3) # The grid points : offset (0,0)
stag = staggered(dx,order_x,bc)

# Check that _dispersion_I is indeed the Fourier transform of the finite differences operator
#def expi(x): return np.exp(1j*(k[0]*x[0]+k[1]*x[1]))
#def dspi(axis): return 1j*_dispersion_I(k[axis],dx,order_x)
#X_00,X_10,X_01,X_11 = shifted_grids(X,dx)
#assert np.allclose(stag.diff_left( expi(X_10),0),     expi(X)*dspi(0))
#assert np.allclose(stag.diff_right(expi(2*X-X_10),0), expi(X)*dspi(0))
#assert np.allclose(stag.diff_left( expi(X_01),1),     expi(X)*dspi(1))
#assert np.allclose(stag.diff_right(expi(2*X-X_01),1), expi(X)*dspi(1))

# Time step
dt = 0.01
Nt = 1
T = Nt*dt

# Dispersion relation and exact solution
ω,vq,vp = dispersion_Virieux(k,ρ,C,dx,dt)
mode = 0
q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])

q0,p0 = eval_Virieux(q_exact,p_exact,X,dx,0,dt)
qf,pf = eval_Virieux(q_exact,p_exact,X,dx,T,dt)

def af(q): return fd.as_field(q,shape=X[0].shape)
scheme = Virieux3(af(ρ),af(C),stag)
q1,p1 = scheme.step(q0,p0,dt)
assert np.allclose(qf,q1)
assert np.allclose(pf,p1)

# Now testing the Lebedev scheme
Q0,P0 = eval_Lebedev(q_exact,p_exact,X,dx,0,dt)
Qf,Pf = eval_Lebedev(q_exact,p_exact,X,dx,T,dt)
assert np.allclose(Qf[0][0],qf[0])
assert np.allclose(Qf[1][1],qf[1])
assert np.allclose(Qf[2][2],qf[2])
assert np.allclose(Pf[0][0],pf[0])
assert np.allclose(Pf[1][1],pf[1])
assert np.allclose(Pf[2][2],pf[2])

Scheme = Lebedev3(af(ρ),af(C),stag)
Q1,P1 = Scheme.step(Q0,P0,dt)
# For a VTI model, the Lebedev scheme consists of 2^(d-1) decoupled Virieux schemes
(dp0_100,dp1_010,dp2_001),(σ00_000,σ11_000,σ22_000,σ01_110,σ02_101,σ12_011),(ϵ00_000,ϵ11_000,ϵ22_000,ϵ01_110,ϵ02_101,ϵ12_011) = scheme.tmp
(dp_100,dp_010,dp_001,dp_111), (σ_000,σ_110,σ_101,σ_011), (ϵ_000,ϵ_110,ϵ_101,ϵ_011) = Scheme.tmp

assert np.allclose(ϵ00_000,ϵ_000[0])
assert np.allclose(dp0_100,dp_100[0])
assert np.allclose(dp1_010,dp_010[1])
assert np.allclose(dp2_001,dp_001[2])
assert np.allclose(P0[0][0],p0[0])
assert np.allclose(Pf[0][0],pf[0])
assert np.allclose(Qf[0][0],Q1[0][0])

assert np.allclose(Qf,Q1)
assert np.allclose(Pf,P1)


# However, the Lebedev scheme also handles general elasticity
C = Hooke.mica[0].rotate_by(π/6,(1,2,3)).hooke; C/=np.max(C); ρ=1.5
ω,vq,vp = dispersion_Virieux(k,ρ,C,dx,dt)
mode = 1
q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])
Q0,P0 = eval_Lebedev(q_exact,p_exact,X,dx,0,dt)
Qf,Pf = eval_Lebedev(q_exact,p_exact,X,dx,T,dt)
Scheme = Lebedev3(af(ρ),af(C),stag)
Q1,P1 = Scheme.step(Q0,P0,dt)
assert np.allclose(Qf,Q1)
assert np.allclose(Pf,P1)
--->

## 5. Numerical dispersion



### 5.1 Isotropic dispersion

Recall that an isotropic Hooke tensor is defined in terms of two Lame parameters $\lambda$ and $\mu$, and satisfies 
$$
    C(m,m) := \sum_{ijkl} C_{ijkl} m_{ij} m_{kl} = \lambda \Tr(m)^2 + 2 \mu \Tr(m^2).
$$
We defined, for any wave number $\kw$, a symmetric matrix $C[\kw]$ as follows
$$
    C[\kw]_{il} := \sum_{jk} \kw_{j}\kw_k,
$$
Noting that $v^\top C[\kw] v = C(m,m)$ with $2 m = \kw v^\top + v \kw^\top$, we obtain the explicit expression is the isotropic case
$$
    C[\kw] = \lambda \kw \kw^\top + \mu(|\kw|^2 \Id + \kw \kw^\top),
$$
which has eigenvalues $\mu |\kw|^2$ and $(\lambda+2\mu)|\kw|^2$, with eigenspaces $\kw^\perp$ and $\kw\bR$. 
Using the dispersion relation $\rho \omega^2 v = C[k] v$, we obtain the associated wave speeds
\begin{align*}
    V_S &= \sqrt{\frac \mu \rho},& V_P =& \sqrt{\frac{\lambda+2 \mu} \rho}.
\end{align*}

A measure of quality of an elastic wave scheme is how well it reproduces these wave speeds, in other words the (lack of) dispersion relation in the isotropic setting.

**Virieux scheme.**
The modified dispersion relation reads
$$
    \rho \rI_2^{\diff t}(\omega) v = C[\rI_{ox}^{\diff x}(\kw)] v
$$

**Selling based scheme**
The effect is more complex, and depends on the sign of $\lambda$, which leads to different Selling decompositions. 
We begin with the two dimensional and second order case, and do not show all intermediate computations. The other cases are treated similarly.
Define 
$$
    E_2^{\diff x}(v) := 
    \begin{pmatrix}
    \rI_2^{\diff x}(v_0)^2 & \rI_2^{2\diff x}(v_0) \rI_2^{2\diff x}(v_1) \\
    \rI_2^{2\diff x}(v_0) \rI_2^{2\diff x}(v_1) & \rI_2^{\diff x}(v_1)^2 
    \end{pmatrix},
$$
which is an approximation of $v v^\top$. Note the step $2 \diff x$ for the off-diagonal coefficients, which are thus less faithful than the diagonal coefficients. The numerical replacement for the dispersion matrix $C[k]$ is obtained as follows : 

- for $\lambda \geq 0$
$$
    2\mu \begin{pmatrix}\rI_2^{\diff x}(k_0)^2 & \\ & \rI_2^{\diff x}(k_1)^2 \end{pmatrix} + \mu E^{\diff x}_2( k_1,k_0) + \lambda E^{\diff x}_2(k)
$$
- for $\lambda\leq 0$ (note that $\lambda \geq -\mu$ by assumption)
$$
2(\mu+\lambda) \begin{pmatrix}\rI_2^{\diff x}(k_0)^2 & \\ & \rI_2^{\diff x}(k_1)^2 \end{pmatrix} + \mu E^{\diff x}_2( k_1,k_0) - \lambda E^{\diff x}_2(-k_0,k_1)
$$


In other words the diagonal coefficients are the same as for the Virieux scheme, but the off-diagonal coefficients are slightly less accurate, corresponding to a twice larger grid step. (The situation is a little bit more complex with anisotropic Hooke tensors.) 

There appears to be a price to handling general Hooke tensors, which is either a multiplication of the unknowns in the Lebedev case, or slightly degraded off-diagonal coefficients in the dispersion relation in the proposed Selling scheme. Attempts to design a Selling staggered grid scheme for general anisotropy and with the same dispersion as Virieux's in the isotropic case were so far infructuous. 
However, the numerical dispersion of the proposed scheme is favorable w.r.t the Lebedev if we adjust the number of points per wavelength so as to take into account the additional Lebedev unknowns. 

In the next cell, we compute the numerical pulsation in terms of a particular wavenumber, for the different numerical schemes.

In [20]:
# Model parameters
λ = -0.4 # 1
μ = 1.
ρ = 1.5
k = np.array((0.5,1))

# Exact speeds
Vs = np.sqrt(μ/ρ); Vp = np.sqrt((λ+2*μ)/ρ)
k_norm = np.linalg.norm(k)
ω_exact = np.array([Vs,Vp])*k_norm

# Discretization parameters
dx = 0.8
dt = dx/1.3
order_x = 2

# Computation of the numerical speeds
C = Hooke.from_Lame(λ,μ).hooke 
ω_Virieux,_,_ = dispersion_Virieux(k,ρ,C,dx,dt,order_x=order_x)
ω_Lebedev,_,_ = dispersion_Virieux(k,ρ,C,dx*np.sqrt(2),dt,order_x=order_x) # dx*sqrt(2) takes into accout 2x density of unknowns
ω_Selling,_,_ = dispersion_e(k,ρ,C,dx,dt,order_x=order_x)


print(f"{ω_exact=}, points per wavelength : {2*π/(k_norm*dx)=}, CFL : {dt/(Vs*dx)=}")
print(f"{ω_Virieux-ω_exact=}")
print(f"{ω_Lebedev-ω_exact=}")
print(f"{ω_Selling-ω_exact=}")

ω_exact=array([0.91287093, 1.15470054]), points per wavelength : 2*π/(k_norm*dx)=7.024814731040726, CFL : dt/(Vs*dx)=0.9421114395319916
ω_Virieux-ω_exact=array([-0.00889369, -0.00191558])
ω_Lebedev-ω_exact=array([-0.02981232, -0.02903098])
ω_Selling-ω_exact=array([-0.00021163, -0.00900189])


However, the dispersion relation features some axis dependent bias, hence values obtained for a specific wavenumber may not be representative. In the next cell, we compute the relation for all possible directions.

**Choice of level set.**
Consider a wave propagating at a speed $V = \omega/\|k\|$. If $\omega=V$ (adimensionized units), then $\|k\|=1$, and therefore the number of points per wavelength is $2\pi/(k\diff x) = 2 \pi/\diff x$  (and the CFL is $\diff t/(V \diff x)$).

In [21]:
# Try changing the following parameters
order_x=2
dx = 1.2
dt = dx/1.8
λ = 0.4 # Try any value >=-1

# No need to change
μ = 1
ρ = 1
C = Hooke.from_Lame(λ,μ).hooke 
Vs = np.sqrt(μ/ρ); Vp = np.sqrt((λ+2*μ)/ρ)

print(f"Points per wavelength {2*π/dx=:.3f}, {order_x=}, {λ/μ=}")
print(f"Blue : Lebedev (worst). Green : Virieux (best, but VTI only). Red : Selling.")

ak = np.linspace(-1.2,1.2)
ks = np.array(np.meshgrid(ak,ak,indexing='ij'))
def af(x): return fd.as_field(x,ks[0].shape)

ω_Virieux,_,_  = dispersion_Virieux(ks,ρ,C,dx,dt,order_x=order_x)
# The next line adjusts dx to take into account the points per wavelength w.r.t Lebedev 
# Multiply dx by 2**(1/2) for 2D, and 2**(2/3) for 3D
ω_Lebedev,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=order_x)
ω_Selling,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=order_x,order_t=2)


fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].set_title("Shear wave dispersions"); ax[0].axis('equal')
ax[0].contour(*ks,ω_Lebedev[0],levels=[Vs],colors='blue')
ax[0].contour(*ks,ω_Virieux[0],levels=[Vs],colors='green')
ax[0].contour(*ks,ω_Selling[0],levels=[Vs],colors='red')
ax[0].add_patch(plt.Circle((0,0),1,fill=False))

ax[1].set_title("Pressure wave dispersions"); ax[1].axis('equal')
ax[1].contour(*ks,ω_Lebedev[1],levels=[Vp],colors='blue')
ax[1].contour(*ks,ω_Virieux[1],levels=[Vp],colors='green')
ax[1].contour(*ks,ω_Selling[1],levels=[Vp],colors='red')
ax[1].add_patch(plt.Circle((0,0),1,fill=False));

Points per wavelength 2*π/dx=5.236, order_x=2, λ/μ=0.4
Blue : Lebedev (worst). Green : Virieux (best, but VTI only). Red : Selling.


### 5.2 VTI and TTI dispersion

In the VTI case, for the mica medium, the Virieux and proposed dispersions are almost identical, whereas the Lebedev scheme is again less accurate.

In [22]:
# Try changing the following parameters
order_x=2
dx = 1.2
dt = dx/1.8
C = Hooke.mica[0].extract_xz().hooke

# No need to change
ρ = 1

print(f"Points per wavelength {2*π/dx=:.3f}, {order_x=}, {λ/μ=}")
print(f"Blue : Lebedev (worst). Green : Virieux (best, but VTI only). Red : Selling.")

ak = np.linspace(-1.2,1.2)
ks = np.array(np.meshgrid(ak,ak,indexing='ij'))
def af(x): return fd.as_field(x,ks[0].shape)

ω_exact,_ = Hooke(C).waves(ks,ρ)
ω_Virieux,_,_  = dispersion_Virieux(ks,ρ,C,dx,dt,order_x=order_x)
ω_Lebedev,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=order_x)
ω_Selling,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=order_x,order_t=2)


fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].set_title("Shear wave dispersions"); ax[0].axis('equal')
ω0 = 0.7*ω_exact[0].mean()
ax[0].contour(*ks,ω_Lebedev[0],levels=[ω0],colors='blue')
ax[0].contour(*ks,ω_Virieux[0],levels=[ω0],colors='green')
ax[0].contour(*ks,ω_Selling[0],levels=[ω0],colors='red')
ax[0].contour(*ks,ω_exact[0],levels=[ω0],colors='black')

ax[1].set_title("Pressure wave dispersions"); ax[1].axis('equal')
ω1 = 0.8*ω_exact[1].mean()
ax[1].contour(*ks,ω_Lebedev[1],levels=[ω1],colors='blue')
ax[1].contour(*ks,ω_Virieux[1],levels=[ω1],colors='green')
ax[1].contour(*ks,ω_Selling[1],levels=[ω1],colors='red')
ax[1].contour(*ks,ω_exact[1],levels=[ω1],colors='black');

Points per wavelength 2*π/dx=5.236, order_x=2, λ/μ=0.4
Blue : Lebedev (worst). Green : Virieux (best, but VTI only). Red : Selling.


Finally, we consider the TTI case. Note that the Virieux scheme does not apply, but that the `dispersion_Virieux` function does, with the correct result for Lebedev as illustrated above. In this setting find that :
- the Lebedev scheme is more accurate for the shear wave.
- the proposed Selling scheme is more accurate for the pressure wave.

In [23]:
# Try changing the following parameters
order_x=4
dx = 1.5
dt = dx/1.8
C = Hooke.mica[0].extract_xz().rotate_by(π/8).hooke
#C = Hooke.stishovite[0].extract_xz().rotate_by(π/8).hooke
C /= np.max(C)

# No need to change
ρ = 1

print(f"Points per wavelength {2*π/dx=:.3f}, {order_x=}, {λ/μ=}")
print(f"Blue : Lebedev. Red : Selling.")

ak = np.linspace(-1.2,1.2)
ks = np.array(np.meshgrid(ak,ak,indexing='ij'))
def af(x): return fd.as_field(x,ks[0].shape)

ω_exact,_ = Hooke(C).waves(ks,ρ)
ω_Lebedev,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=order_x)
ω_Selling,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=order_x,order_t=2)
#ω_Selling,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=6,order_t=2) # Order 6 does not bring much benefit


fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].set_title("Shear wave dispersions"); ax[0].axis('equal')
ω0 = 0.7*ω_exact[0].mean()
ax[0].contour(*ks,ω_Lebedev[0],levels=[ω0],colors='blue')
ax[0].contour(*ks,ω_Selling[0],levels=[ω0],colors='red')
ax[0].contour(*ks,ω_exact[0],levels=[ω0],colors='black')

ax[1].set_title("Pressure wave dispersions"); ax[1].axis('equal')
ω1 = 0.8*ω_exact[1].mean()
ax[1].contour(*ks,ω_Lebedev[1],levels=[ω1],colors='blue')
ax[1].contour(*ks,ω_Selling[1],levels=[ω1],colors='red')
ax[1].contour(*ks,ω_exact[1],levels=[ω1],colors='black');

Points per wavelength 2*π/dx=4.189, order_x=4, λ/μ=0.4
Blue : Lebedev. Red : Selling.


The decompostion of the Hooke tensor of this TTI medium, which is rather strongly anisotropic and in general orientation, involves several non-trivial, anisotropic offsets, which are already 'quite large'. This appears to be required to express the elastic energy in the desired sum of squares form. However, it also means that the effective resolution of the scheme is decreased (as in all 'wide stencil' methods), and therefore the dispersion relation is a bit degraded. 

In [24]:
Hooke(C).Selling()

(array([0.05112568, 0.01688454, 0.15667604, 0.15667604, 0.36400569, 0.2072774 ]),
 array([[[ 2, -2,  1,  0,  1, -1],
         [ 1, -1,  1,  0,  0,  0]],
 
        [[ 1, -1,  1,  0,  0,  0],
         [ 0, -1,  0,  1,  0, -1]]]))

The Hooke tensor of this model, in its rotation invariant Mandel form, has a condition number $\approx 9$. 

In [25]:
np.linalg.eigvalsh(Hooke(C).to_Mandel())

array([0.17356058, 0.37852637, 1.27812355])

When the medium is less strongly anisotropic, for instance stishovite here, the Selling scheme regains advantage.

In [26]:
# Try changing the following parameters
order_x=4
dx = 2
dt = dx/2
C = Hooke.stishovite[0].extract_xz().rotate_by(π/8).hooke
C /= np.max(C)

# No need to change
ρ = 1

print(f"Points per wavelength {2*π/dx=:.3f}, {order_x=}, {λ/μ=}")
print(f"Blue : Lebedev. Red : Selling.")

ak = np.linspace(-1.2,1.2)
ks = np.array(np.meshgrid(ak,ak,indexing='ij'))
def af(x): return fd.as_field(x,ks[0].shape)

ω_exact,_ = Hooke(C).waves(ks,ρ)
ω_Lebedev,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=order_x)
ω_Selling,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=order_x,order_t=2)
#ω_Selling,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=6,order_t=2) # Order 6 does not bring much benefit

fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].set_title("Shear wave dispersions"); ax[0].axis('equal')
ω0 = 0.7*ω_exact[0].mean()
ax[0].contour(*ks,ω_Lebedev[0],levels=[ω0],colors='blue')
ax[0].contour(*ks,ω_Selling[0],levels=[ω0],colors='red')
ax[0].contour(*ks,ω_exact[0],levels=[ω0],colors='black')

ax[1].set_title("Pressure wave dispersions"); ax[1].axis('equal')
ω1 = 0.8*ω_exact[1].mean()
ax[1].contour(*ks,ω_Lebedev[1],levels=[ω1],colors='blue')
ax[1].contour(*ks,ω_Selling[1],levels=[ω1],colors='red')
ax[1].contour(*ks,ω_exact[1],levels=[ω1],colors='black');

Points per wavelength 2*π/dx=3.142, order_x=4, λ/μ=0.4
Blue : Lebedev. Red : Selling.


In [27]:
Hooke(C).Selling()

(array([0.06151716, 0.04518867, 0.19772098, 0.76074813, 0.39544195, 0.1940632 ]),
 array([[[-1,  1,  0,  0, -1,  1],
         [ 1, -1,  1,  0,  0,  0]],
 
        [[ 1, -1,  1,  0,  0,  0],
         [ 0,  1,  0, -1,  0,  1]]]))

## 6. Correlated Selling scheme

For the shear wave in the mica material, the Selling scheme presented above is a bit disappointing in terms of dispersion. In this section and the next one, we explore scheme variants intended to improve on this.

The Selling scheme involves sums of squares of finite differences such as 
$$
	\sum_{\sigma\in \{-1,1\}^d} 
	\Big(\sum_{1\leq i \leq d} \frac{u_i(x+h \sigma_i e_i)-u_i(x)}{h \sigma_i}\Big)^2.
$$

### 6.1 Scheme implementation

By choosing only a subset of the possible signs, in the above sum, optimize the numerical dispersion. 
Indeed two derivatives in approximately the same direction approximately amounts a second derivative in that direction, in which case the dispersion is smaller.

For instance, if $e_1=e_2$ in dimension $2$, then one can consider the constraints $\sigma_1=\sigma_2$. In contrast, if $e_1=-e_2$ the we may consider the constraint $\sigma_1=-\sigma_2$. Likewise if $e_1$ and $e_2$ are approximately collinear. 

In [28]:
def SellingCorrelated2H(ρ,C,X,dx,order_x=2,bc='Periodic'):
    λ,E = C if isinstance(C,tuple) else Hooke(C).Selling() # Note that E is a (collection of) symmetric matrices
    corr = lp.dot_VV(E[0],E[1]) # Correlation between the two offsets (do they point in the same direction)
    λ,corr = [fd.as_field(e,X[0].shape) for e in (λ,corr)]
    padding = AnisotropicWave.bc_to_padding[bc]
    def PotentialEnergy(q):
        dq0 = fd.DiffEll(q[0],E[0],dx,order=order_x,padding=padding)
        dq1 = fd.DiffEll(q[1],E[1],dx,order=order_x,padding=padding)
        sq_pos = np.sum((dq0+dq1      )**2,axis=0) # Best when positive correlation
        sq_neg = np.sum((dq0+dq1[::-1])**2,axis=0) # Best when negative correlation
        return 0.5 * λ * np.where(corr==0,(sq_pos+sq_neg)/2, # Symmetric version
                         np.where(corr>0,  sq_pos,sq_neg  )) # Asymmetric versions
    def KineticEnergy(p): return 0.5*p**2/ρ
    H = QuadraticHamiltonian(PotentialEnergy,KineticEnergy); H.set_spmat(np.zeros_like(X)); return H

This following slightly generalized variant accepts the coefficients obtained through changes of variables, see the notebook on [high order schemes for the wave equations](HighOrderWaves.ipynb).

In [112]:
def SellingCorrelated2H_ext(M,C,X,dx,order_x=2,S=None,bc='Periodic'):
    λ,E = C if isinstance(C,tuple) else Hooke(C).Selling() # Note that E is a (collection of) symmetric matrices
    corr = lp.dot_VV(E[0],E[1]) # Correlation between the two offsets (do they point in the same direction)
    λ,corr = [fd.as_field(e,X[0].shape) for e in (λ,corr)]
    padding = AnisotropicWave.bc_to_padding[bc]
    if S is None: ES = (0.,0.)
    else: ES = np.sum(E[:,:,None,:]*S[:,:,:,None],axis=(0,1))
    def PotentialEnergy(q):
        dq0 = fd.DiffEll(q[0],E[0],dx,order=order_x,α=ES[0]*q[0],padding=padding)
        dq1 = fd.DiffEll(q[1],E[1],dx,order=order_x,α=ES[1]*q[1],padding=padding)
        sq_pos = np.sum((dq0+dq1      )**2,axis=0) # Best when positive correlation
        sq_neg = np.sum((dq0+dq1[::-1])**2,axis=0) # Best when negative correlation
        return 0.5 * λ * np.where(corr==0,(sq_pos+sq_neg)/2, # Symmetric version
                         np.where(corr>0,  sq_pos,sq_neg  )) # Asymmetric versions
    def KineticEnergy(p): return 0.5*p[None,:]*p[:,None]*M
    H = QuadraticHamiltonian(PotentialEnergy,KineticEnergy); H.set_spmat(np.zeros_like(X)); return H

### 6.2 Dispersion relation

We have the following observations regarding dispersion : 

- There is a substantial benefit for the second order Selling scheme, especially on the numerical dispersion of the shear wave. In some cases, the second order Selling scheme may become as good as or better than the fourth order Selling scheme. Accuracy becomes better than the second order Lebedev scheme.
- There appears to be *almost no benefit* for the fourth and sixth order schemes.

In summary, this modification appears to be beneficial only when the second order Selling scheme is used with strongly anisotropic media (crystals, at least). Implementation will be done if there is a good use case.

In [29]:
from agd.ExportedCode.Notebooks_Div.HighOrderWaves import _dispersion

def sin_eet(e,dx,order_x=2,sign=0):
    """Approximates of e e^T as dx->0, arises from FFT of scheme."""
    def ondiag(s): # Fourier transform of finite difference approximation of second derivative, see _γ
        if order_x==2: return 4*np.sin(s/2)**2
        if order_x==4: return (np.cos(2*s)-16*np.cos(s)+15)/6
        if order_x==6: return 49/18-3*np.cos(s)+3/10*np.cos(2*s)-1/45*np.cos(3*s)
    def offdiag_sym(s,t): # Fourier transform of symmetrized finite difference approximation of cross derivative
        if order_x==2: return np.sin(s)*np.sin(t)
        if order_x==4: return (8*np.sin(s)*np.sin(t) + (4*np.sin(s)-np.sin(2*s))*(4*np.sin(t)-np.sin(2*t)))/12
        if order_x==6: return (-9*np.sin(2*s)*(13*np.sin(t)-5*np.sin(2*t)+np.sin(3*t))+9*np.sin(s)*(50*np.sin(t)-13*np.sin(2*t)+2*np.sin(3*t))+np.sin(3*s)*(18*np.sin(t)-9*np.sin(2*t)+2*np.sin(3*t)))/180        
    def offdiag_asym(s,t): # Fourier transform of non-symmetrized finite difference approximation of cross derivative
        if order_x==2: return 1 - np.cos(s) + np.cos(s - t) - np.cos(t)
        if order_x==4: return -(1/12)*(-9+12*np.cos(s)-3*np.cos(2*s)+4*np.cos(s-2*t)-20*np.cos(s-t)-np.cos(2*(s-t))+4*np.cos(2*s-t)+12*np.cos(t)-3*np.cos(2*t)+4*np.cos(s+t))
        if order_x==6: return (1/180)*(101-153*np.cos(s)+63*np.cos(2*s)-11*np.cos(3*s)+18*np.cos(s-3*t)-9*np.cos(2*s-3*t)-108*np.cos(s-2*t)-9*np.cos(3*s-2*t)+342*np.cos(s-t)+45*np.cos(2*(s-t))+2*np.cos(3*(s-t))-108*np.cos(2*s-t)+18*np.cos(3*s-t)-153*np.cos(t)+63*np.cos(2*t)-11*np.cos(3*t)-108*np.cos(s+t)+9*np.cos(2*s+t)+9*np.cos(s+2*t))
    def offdiag(s,t,σ): # Fourier transform of finite difference approximation of cross derivative
        return offdiag_sym(s,t)-σ*(offdiag_sym(s,t)-offdiag_asym(s,t))
    vdim = len(e)
    return np.array([[ondiag(e[i]*dx) if i==j else offdiag(e[i]*dx,e[j]*dx,sign if vdim==2 else sign[3-i-j])
             for i in range(vdim)] for j in range(vdim)])/dx**2

def corr_signs(σ):
    """
    Input : a set of vdim = 2 or 3 vectors.
    Ouput : sums of best correlations σi σj, 0 <= i < j < vdim.
    """
    if len(σ)==2: return np.sign(lp.dot_VV(σ[:,0],σ[:,1])) # Best sign correlation in dimension 2
    # See discussion below about best sign correlations in dimension 3
    σs = np.array([lp.dot_VV(σ[:,1],σ[:,2]),lp.dot_VV(σ[:,0],σ[:,2]),lp.dot_VV(σ[:,0],σ[:,1])])
    ϵ = ((1,1,1),(-1,1,1),(1,-1,1),(-1,-1,1))
    ϵs = np.array([(ϵ1*ϵ2,ϵ0*ϵ2,ϵ0*ϵ1) for ϵ0,ϵ1,ϵ2 in ϵ])
    ϵs = np.expand_dims(ϵs,axis=tuple(range(-σ.ndim+2,0)))
    σϵs = np.sum(σs*ϵs,axis=1)
    σϵmax = σϵs==np.max(σϵs,axis=0)
    return np.sign(np.sum(ϵs*σϵmax[:,None],axis=0))

def sin_contract(C,k,dx,order_x):
    """Approximates Hooke(C).contract(k) as dx->0, arises from FFT of scheme."""
    λ,σ = C if isinstance(C,tuple) else Hooke(C).Selling()
    σk = lp.dot_AV(σ,k[:,None])
    σs = corr_signs(σ)
    return np.sum(λ*sin_eet(σk,dx,order_x,σs),axis=2)
    
def dispersion_SellingCorrelated(k,ρ,C,dx,dt,order_x=2):
    """Return all discrete propagation modes."""
    # For now, we assume that M = Id/ρ. Anisotropic M seem doable, but would require taking care
    # of the non-commutativity of a number of matrices in the scheme.
    from agd.ExportedCode.Notebooks_Div.HighOrderWaves import eig
    Ck = sin_contract(C,k,dx,order_x)
    return _dispersion_ElasticT2(ρ,Ck,dt)

Let us now cross validate the scheme with the numerical dispersion relation.

In [30]:
C_iso = Hooke.from_Lame(1.,1.).hooke
C_VTI2 = Hooke.mica[0].extract_xz().hooke
C_TTI2 = Hooke.mica[0].extract_xz().rotate_by(π/6).hooke
for C in (C_VTI2,C_TTI2): C/=np.max(C)

for ρ, C, k, order_x, Nx, dt, mode in [
    (1.2, C_iso,  [ 1, 1], 2, 10, 0.01, 0),
    (1.2, C_VTI2, [ 1,-2], 4, 9,  0.03, 1),
    (1.4, C_TTI2, [ 2, 3], 2, 12, 0.05, 0),
    (0.9, C_TTI2, [-1, 1], 6, 7,  0.02, 1),
]:
    vdim = len(k)
    k = 2*π*np.array(k)
    X, dx = make_domain(Nx,vdim) # The grid points : offset (0,0)

    # Time discretization
    Nt = 1
    T = Nt*dt

    # Dispersion relation and exact solution
    ω,vq,vp = dispersion_SellingCorrelated(k,ρ,C,dx,dt,order_x)
    q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])

    q0,p0 = q_exact(  dt/2,X),p_exact(0,X)
    qf,pf = q_exact(T+dt/2,X),p_exact(T,X)

    SellingCorrH = SellingCorrelated2H(ρ,C,X,dx,order_x)
    q1,p1 = SellingCorrH.Euler_p(q0,p0,dt)
    assert np.allclose(pf,p1)
    assert np.allclose(qf,q1)

### 6.3 Visualizing dispersion

In [31]:
# Try changing the following parameters
dx = 1.
dt = dx/1.8
C = Hooke.mica[0].extract_xz().rotate_by(π/6).hooke
#C = Hooke.stishovite[0].extract_xz().rotate_by(π/8).hooke
C /= np.max(C)

# No need to change
ρ = 1

ak = np.linspace(-1.2,1.2)
ks = np.array(np.meshgrid(ak,ak,indexing='ij'))
def af(x): return fd.as_field(x,ks[0].shape)

ω_exact,_ = Hooke(C).waves(ks,ρ)
ω_Lebedev2,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=2)
ω_Lebedev4,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=4)

ω_Selling2,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=2,order_t=2)
ω_Selling4,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=4,order_t=2)
ω_Selling6,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=6,order_t=2)
ω_Selling2_corr,_,_  = dispersion_SellingCorrelated(ks,af(ρ),af(C),dx,dt,order_x=2)
ω_Selling4_corr,_,_  = dispersion_SellingCorrelated(ks,af(ρ),af(C),dx,dt,order_x=4)
ω_Selling6_corr,_,_  = dispersion_SellingCorrelated(ks,af(ρ),af(C),dx,dt,order_x=6)

fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].set_title("Shear wave dispersion"); ax[0].axis('equal')
ω0 = 0.7*ω_exact[0].mean()
ax[0].contour(*ks,ω_Lebedev2[0],levels=[ω0],colors='blue')
ax[0].contour(*ks,ω_Lebedev4[0],levels=[ω0],colors='cyan')
ax[0].contour(*ks,ω_Selling2[0],levels=[ω0],colors='red')
ax[0].contour(*ks,ω_Selling4[0],levels=[ω0],colors='orange')
ax[0].contour(*ks,ω_Selling2_corr[0],levels=[ω0],colors='green')
ax[0].contour(*ks,ω_Selling4_corr[0],levels=[ω0],colors='olive')
ax[0].contour(*ks,ω_exact[0],levels=[ω0],colors='black')

ax[1].set_title("Pressure wave dispersion"); ax[1].axis('equal')
ω1 = 0.8*ω_exact[1].mean()
ax[1].contour(*ks,ω_Lebedev2[1],levels=[ω1],colors='blue')
ax[1].contour(*ks,ω_Lebedev4[1],levels=[ω1],colors='cyan')
ax[1].contour(*ks,ω_Selling2[1],levels=[ω1],colors='red')
ax[1].contour(*ks,ω_Selling4[1],levels=[ω1],colors='orange')
ax[1].contour(*ks,ω_Selling2_corr[1],levels=[ω1],colors='green')
ax[1].contour(*ks,ω_Selling4_corr[1],levels=[ω1],colors='olive')
ax[1].contour(*ks,ω_exact[1],levels=[ω1],colors='black');

print(f"Points per wavelength {2*π/dx=:.3f}")
print(f"Blue : Lebedev. Red : Selling. Green : correlated Selling")

Points per wavelength 2*π/dx=6.283
Blue : Lebedev. Red : Selling. Green : correlated Selling


In [32]:
C = Hooke.mica[0].extract_xz().rotate_by(0.3).hooke; C /= np.max(C)
Hooke(C).Selling()

(array([0.08383371, 0.00447446, 0.0702737 , 0.16166576, 0.45269118, 0.17790585]),
 array([[[ 1, -1,  2,  0, -1,  1],
         [ 1, -1,  1,  0,  0,  0]],
 
        [[ 1, -1,  1,  0,  0,  0],
         [ 0,  1,  0, -1,  0,  1]]]))

### 6.4 Three dimensional case

In three or more dimensions, correlating the offsets means maximizing 
$$
    \| \sum_{1 \leq i \leq I} \sigma_i e_i\|^2 = \sum_{1 \leq i \leq I} \|e_i\|^2 + 2\sum_{i<j} \sigma_i \sigma_j \<e_i,e_j\>,
$$
over all possible sign choices $(\sigma_i)_{1 \leq i \leq I}$, $\sigma_i = \pm 1$.
This amounts to maximizing a quadratic form over boolean variables. The problem is known to be np-complete in high dimension, but with only three unknowns it does not raise any specific difficulty.

For symmetry, we sum over all maximal sign combinations. Some notes : 
- Heuristically, for 'most' offsets, there should be only one maximal combination. However, in practice, offsets are small, integer valued, and have many zeros. Hence orthogonality is likely. 
- Regarding the CPU implementation, we make a weighted sum over all combinations, and let the sparse matrix construction do the rest.
- The GPU implementation is *todo*.

Regarding invariances : 
- $(\sigma_0,\sigma_1,\sigma_2)\in \{-1,1\}^3$ is equivalent to $(-\sigma_0,-\sigma_1,-\sigma_2)$. Therefore one may assume $\sigma_2=1$. Alternatively, one may look for a combination among $(+,+,+)$, $(+,+,-)$, $(+,-,+)$, $(-,+,+)$.
- blabla

In [33]:
σ0 = np.array([[1,0],[0,1],[-1,-1]]).T # Best correlation (++-) -> products (--+)
σ1 = np.array([[1,0],[0,0],[-1,-1]]).T # (++-) and (+--) -> (0-0)
σ2 = np.array([[1,0],[0,1],[1,-1]]).T  # (+-+) -> (-+-)
σ=np.stack((σ0,σ1,σ2),axis=-1)
corr_signs(σ) 

array([0, 0, 0])

In [34]:
def SellingCorrelatedH(ρ,C,X,dx,order_x=2,bc='Periodic'):
    λ,E = C if isinstance(C,tuple) else Hooke(C).Selling() # Note that E is a (collection of) symmetric matrices
    ϵ = corr_signs(E) # Correlation between the two offsets (do they point in the same direction)
    λ,ϵ = [fd.as_field(e,X[0].shape) for e in (λ,ϵ)]
    ϵ_pos = (ϵ>0)+(ϵ>=0).astype(int); ϵ_neg = (ϵ<0)+(ϵ<=0).astype(int) # Caution : array([True])+array([True]) == array([True])
    
    padding = AnisotropicWave.bc_to_padding[bc]
    def PotentialEnergy(q):
        dq = ad.array([fd.DiffEll(qi,Ei,dx,order=order_x,padding=padding) for qi,Ei in zip(q,E)])
        if vdim==2: return 0.5 * λ * (dq[0]**2+dq[1]**2 + ϵ_pos*dq[0]*dq[1] + ϵ_neg*dq[0]*dq[1,::-1])
        dq_pos = ad.array([dq[1]*dq[2],     dq[0]*dq[2],     dq[0]*dq[1]])
        dq_neg = ad.array([dq[1]*dq[2,::-1],dq[0]*dq[2,::-1],dq[0]*dq[1,::-1]])
        return 0.5 * λ * (dq**2 + ϵ_pos[:,None]*dq_pos + ϵ_neg[:,None]*dq_neg)
    def KineticEnergy(p): return 0.5*p**2/ρ
    H = QuadraticHamiltonian(PotentialEnergy,KineticEnergy); H.set_spmat(np.zeros_like(X)); return H

Let us cross validate with the dispersion relation.

In [35]:
C_iso2 = Hooke.from_Lame(2.,1.).hooke
C_VTI2 = Hooke.mica[0].extract_xz().hooke
C_TTI2 = Hooke.mica[0].extract_xz().rotate_by(π/6).hooke
C_iso3 = Hooke.from_Lame(1.,1.,vdim=3).hooke
C_VTI3 = Hooke.mica[0].hooke
C_TTI3 = Hooke.mica[0].rotate_by(π/6,(1,2,3)).hooke

for C in (C_VTI2,C_TTI2,C_VTI3,C_TTI3): C/=np.max(C)

for ρ, C, k, order_x, Nx, dt, mode in [
    (1.2, C_iso2, [ 1, 1], 2, 10, 0.01, 0),
    (1.2, C_VTI2, [ 1,-2], 4, 9,  0.03, 1),
    (1.4, C_TTI2, [ 2, 3], 6, 12, 0.05, 0),
    (0.8, C_iso3, [-1, 1, 1], 2, 7,  0.02, 1),
    (1.2, C_VTI3, [ 1,-2,-1], 4, 9,  0.03, 1),
    (1.4, C_TTI3, [ 2, -1,3], 2, 12, 0.05, 0),
]:
    vdim = len(k)
    k = 2*π*np.array(k)
    X, dx = make_domain(Nx,vdim) # The grid points : offset (0,0)

    # Time discretization
    Nt = 1
    T = Nt*dt

    # Dispersion relation and exact solution
    ω,vq,vp = dispersion_SellingCorrelated(k,ρ,C,dx,dt,order_x)
    q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])

    q0,p0 = q_exact(  dt/2,X),p_exact(0,X)
    qf,pf = q_exact(T+dt/2,X),p_exact(T,X)

    SellingCorrH = SellingCorrelatedH(ρ,C,X,dx,order_x)
    q1,p1 = SellingCorrH.Euler_p(q0,p0,dt)
    assert np.allclose(pf,p1)
    assert np.allclose(qf,q1)

## 7. Staggered Selling scheme

A key strength of the Virieux scheme is its very efficient evaluation of the term 
$$
    \partial^1 q^0 + \partial^0 q^1,
$$
thanks to the use of a staggered grid.

By default, the Selling scheme is not compatible with a staggered grid, because it uses arbitrary matrix offsets. 
The offsets (converted in symmetric 2x2 matrix form through Voigt notation) compatible with the staggered grid are 
\begin{align*}
    &\begin{pmatrix}
    \alpha & 0\\
    0 & \beta
    \end{pmatrix}&
    &\begin{pmatrix}
    a & c \\
    c & b
    \end{pmatrix}&
\end{align*}
where $\alpha,\beta \in \bR$ are arbitrary, and $a,b,c \in \bZ$ are integers such that $a\equiv b \, \modtwo$ and $a\not \equiv c\,\modtwo$.

Indeed, we have a staggered finite differences scheme for 
$$
    \<\nabla q^0,(a,c)\> + \<\nabla q^1,(c,b)\>,
$$
which is discretized at the position $(0,0)$ if $c\equiv 0\, \modtwo$, or $(1/2,1/2)$ if $c\equiv 1\, \modtwo$.

### 7.1 Staggered Selling decomposition

Selling's decomposition is defined as a linear program, with infinitely many unknowns corresponding to all possible offsets. 
We would like a variant that only uses the above offsets. So far, we cannot do this in full generality, or in a way fast enough for practical uses. But as a first experiment, we can select a few offsets of small norm.
We could use a generic linear solver to find some decomposition, but for efficiency and reproducibility we prefer to precompute the problem structure. 

The objective is to cover the majority of geological materials, as catalogued by Thomsen. In the implementation below, we allow the following offsets in the Hooke tensor decomposition : $(1,0,0),(0,1,0),(0,0,1), (2,0,1),(2,2,1)$. These triplets correspond to $(a,b,c)$ in the previous paragraph, and sign changes $(\pm a, \pm b,c)$ as well as interversions $(\pm b,\pm a,c)$ of the first two coordinates, are also allowed. 
The last two are absent from the original Virieux scheme, and allow to cover all Thomsen anisotropies except the biotite crystal.

<!---
_staggered_decomp_offsets = np.moveaxis([(1,0,0),(0,1,0), # In solver, prepend additional off diagonal to address (α,β,0)
    (0,0,1), (2,0,1),(-2,0,1),(0,2,1),(0,-2,1), (2,2,1),(-2,2,1),(2,-2,1),(-2,-2,1)],-1,0) # Works for every Thomsen except biotite crystal                              

def staggered_decomp_linprog(m):
    """A variant of the Selling/Voronoi decomposition, compatible with staggered grids"""
    assert m.ndim==2 # TODO : apply to a family of matrices. Decomposition is done pointwise.
#    if m.ndim>2: return np.reshape([staggered_decomp_linprog(am.reshape((3,3,-1)) ....
    
    # Setup and solve the linear program
    flat_sym = flatten_symmetric_matrix
    A_eq = flat_sym(lp.outer_self(_staggered_decomp_offsets))
    offdiag = np.array([0,1,0,0,0,0])[:,None]
    A_eq = np.concatenate((offdiag,A_eq),axis=1)
    b_eq = flat_sym(m)
    c = -np.ones(A_eq.shape[1])
    c[0]=0
    bounds = [(None,None),*tuple(((0,None),))*(len(c)-1)]
    res = linprog(c,A_eq=A_eq,b_eq=b_eq,bounds=bounds)
    if not res.success: return None,None,None,False
    coefs_00 = res.x[:3]
    coefs_11 = res.x[3:]
    offsets_11 = _staggered_decomp_offsets[:,2:]
    
    # Validate result, and extract decomp. Note that the second criterion is observed but was not enforced.
    valid = res.success and coefs_00[1]*coefs_00[2]>=coefs_00[0]**2
    arg = np.argsort(coefs_11)
    ncoefs_11 = 6-np.sum(coefs_00>0) # 6 non-zero coefficients overall (d.o.f of m) 
    assert np.allclose(coefs_11[arg[:-ncoefs_11]],0)
    coefs_11 = coefs_11[arg[-ncoefs_11:]]
    offsets_11 = offsets_11[:,arg[-ncoefs_11:]]
    return coefs_00,coefs_11,offsets_11,valid


staggered_decomp_test = [Hooke.from_ThomsenElastic(tem) for name,tem in Seismic.Thomsen.ThomsenData.items() if name!='Biotite crystal'] \
    + [Hooke.mica,Hooke.stishovite,Hooke.olivine]
staggered_decomp_test = [hk[0].extract_xz().hooke for hk in staggered_decomp_test]

nθ = 100; θs = np.linspace(0,π/2,nθ)
print(f"Decomposition of {len(staggered_decomp_test)} materials, over {len(θs)} regular orientations each")

for m in staggered_decomp_test:
    m = m/np.max(m) # Normalize coefficients (optional)
    for θ in θs:
        m_rot = Hooke(m).rotate_by(θ).hooke
        (c_00,a_00,b_00),coefs_11,offsets_11,valid = staggered_decomp_linprog(m_rot)
        assert valid
        rec_00 = np.array([[a_00,c_00,0],[c_00,b_00,0],[0,0,0]])
        rec_11 = np.sum(coefs_11*lp.outer_self(offsets_11),axis=2)
        assert np.allclose(m_rot,rec_00+rec_11)
--->

In [36]:
_precomp_offsets = np.array([[1,0,0,2,0,2],[0,1,0,0,2,2],[0,0,1,1,1,1]])
_precomp_inv = np.linalg.inv(flatten_symmetric_matrix(lp.outer_self(_precomp_offsets)))

def staggered_decomp_linprog(D,select='Cross'):
    """
    Modified Selling decomposition, for moderate anisotropy (all Thomsen materials execpt biotite crystal).
    - select ('Mean','Cross','Short'): selection principle for the decomposition
       'Mean' : average of all possible decompositions
       'Cross' : minimize the cross derivative coefficient, so as to achieve coef01**2 <= coef[0]*coef[1] when possible
       'Short' : minimize the coefficient of the longest offset, so as to improve stencil locality
    """
    # Single vertex, no need to find the best one in Ryskov's polyhedron.
    assert D.shape[:2]==(3,3)
    def af(x): return fd.as_field(x,D.shape[2:]) #np.expand_dims(x,tuple(range(2-D.ndim,0)))
    sdim = D.ndim-2 # number of additional dimensions
    D_flat = flatten_symmetric_matrix(D)
    signs = np.sign(D_flat[3:5])
    signs[signs==0]=1
    D_flat[3:5]*=signs
    if D_flat.ndim<=2: coefs = _precomp_inv@D_flat # Account for weird @ semantics
    else: coefs = np.moveaxis(_precomp_inv@np.moveaxis(D_flat,0,-2),-2,0)
    tmin = - np.min(coefs[[2,5]],axis=0)
    tmax =   np.min(coefs[[3,4]],axis=0)
    # We should have tmin<=tmax, up to machine precision, if the linear program is solvable. 
    # Possible selection principles for parameter t such that tmin<=t<=tmax : 
    if select=='Mean': t = (tmin+tmax)/2 # - midpoint, average of possible decompositions
    else: # Cross : minimize |coef01|. Short : minimize the last coefficient
        if select=='Cross': tmix = np.prod(signs,axis=0)*D[0,1]/4-coefs[-1]
        elif select=='Short': tmix = -coefs[-1]
        t = np.maximum(tmin,np.minimum(tmax,tmix))
    coefs += t*af(np.array([0,0,1,-1,-1,1.]))
    offsets = af(_precomp_offsets).copy()
    offsets[0:2]*=signs[:,None].astype(int)
    # Adjust for the coefficient at (O,1)
    coef01 = D[0,1] - np.sum(np.prod(offsets[0:2],axis=0)*coefs,axis=0)
    return (coefs[0],coefs[1],coef01),coefs[2:],offsets[:,2:]

In [37]:
D0 = Hooke.from_Lame(1,1).hooke
D1 = Hooke.from_Lame(1,2).hooke
D = np.stack((D0,D1),axis=-1)
coefs_00,coefs_11,offsets_11 = staggered_decomp_linprog(D)
np.array(coefs_00).shape,coefs_11.shape,offsets_11.shape

((3, 2), (4, 2), (3, 4, 2))

Because we use only a few offsets, the above decomposition method only applies to moderate anisotropies. Fortunately, this is enough to handle most geological materials (except the biotite crystal).

In [38]:
staggered_decomp_test = [Hooke.from_ThomsenElastic(tem) for name,tem in Seismic.Thomsen.ThomsenData.items() if name!='Biotite crystal'] \
    + [Hooke.mica,Hooke.stishovite,Hooke.olivine]
staggered_decomp_test = [hk[0].extract_xz().hooke for hk in staggered_decomp_test]

nθ = 100; θs = np.linspace(0,π/2,nθ)
print(f"Decomposition of {len(staggered_decomp_test)} materials, over {len(θs)} regular orientations each")

for m in staggered_decomp_test:
    m = m/np.max(m) # Normalize coefficients (optional)
    for θ in θs:
        m_rot = Hooke(m).rotate_by(θ).hooke
        (a_00,b_00,c_00),coefs_11,offsets_11 = staggered_decomp_linprog(m_rot)
        rec_00 = np.array([[a_00,c_00,0],[c_00,b_00,0],[0,0,0]])
        rec_11 = np.sum(coefs_11*lp.outer_self(offsets_11),axis=2)
        assert np.allclose(m_rot,rec_00+rec_11)

Decomposition of 60 materials, over 100 regular orientations each


### 7.2 Scheme implementation


In [39]:
def SellingStaggered2H(ρ,C,stag,X,S=None):
    """Hamiltonian of the 2d elastic wave equation, implemented using a staggered variant of Selling's decomposition."""
    dro,dl,ar = stag.diff_right_offset,stag.diff_left,stag.avg_right
    coefs_00,coefs_11,offsets_11 = staggered_decomp_linprog(C)
    def af(x): return fd.as_field(x,X[0].shape)
    iρ = af(1/ρ)    
    iρ_10 = ar(iρ,0)
    iρ_01 = ar(iρ,1)
    # TODO : higher spatial accuracy by properly locating coefs_11 (average with neighbors)
    assert S is None
    def PotentialEnergy(q):
        q0_10,q1_01 = q
        ϵ00_00 = dl(q0_10,0) # These terms handled similarly to Virieux
        ϵ11_00 = dl(q1_01,1) # Compute the strain tensor ϵ = (Dq+Dq^T)/2
        res = coefs_00[0]*ϵ00_00**2 + coefs_00[1]*ϵ11_00**2 +  2*coefs_00[2]*ϵ00_00*ϵ11_00
        for λ_11,offset_11 in zip(coefs_11,np.moveaxis(offsets_11,1,0)):
            e0,e1 = (offset_11[0],offset_11[2]),(offset_11[2],offset_11[1])
            ϵ_11 = dro(q0_10,1,e0) + dro(q1_01,0,e1)
            res += λ_11 * ϵ_11**2
        return 0.5*res
    def KineticEnergy(p):
        p0_10,p1_01 = p
        return 0.5*(iρ_10*p0_10**2 + iρ_01*p1_01**2)
    H = QuadraticHamiltonian(PotentialEnergy,KineticEnergy); H.set_spmat(np.zeros_like(X)); return H

### 7.3 Dispersion relation

In [40]:
def dispersion_SellingStaggered2(k,ρ,C,dx,dt,order_x=2,decomp=staggered_decomp_linprog):
    """
    Dispersion relation for the staggered Selling scheme for the 2d elastic wave equation.
    Returns. 
    - ω,vq,vp : pulsation, and amplitude of position and impulsion, of the wavemodes.
    """
    coefs_00,coefs_11,offsets_11 = decomp(C)
    def disp_stag(k): return _dispersion_DiffStaggered(k,dx,order_x)
    Ik = disp_stag(k)
    def af(x): return fd.as_field(x,k[0].shape)
    eye = af(np.eye(2))
    Ck_00 = coefs_00[0] * Ik[0]**2 * lp.outer_self(eye[0]) + coefs_00[1] * Ik[1]**2 * lp.outer_self(eye[1]) \
     + coefs_00[2]*Ik[0]*Ik[1]*(lp.outer(eye[0],eye[1])+lp.outer(eye[1],eye[0])) 
    Ck_11 = []
    for λ_11,offset_11 in zip(coefs_11,np.moveaxis(offsets_11,1,0)):
        e0,e1 = (offset_11[0],offset_11[2]),(offset_11[2],offset_11[1])
        e0,e1 = map(af,(e0,e1))
        Ik_11 = disp_stag(lp.dot_VV(k,e0)),disp_stag(lp.dot_VV(k,e1))
        Ck_11.append( λ_11 * lp.outer_self(Ik_11))
    return _dispersion_ElasticT2(ρ,Ck_00+np.sum(Ck_11,axis=0),dt)

The staggered Selling scheme reduces to the Virieux scheme in the VTI setting, and so does its dispersion.

In [41]:
ρ = 1.4
C = Hooke.mica[0].extract_xz().hooke
#C = Hooke.stishovite[0].extract_xz().hooke
#C = Hooke.from_Lame(1,2).hooke; C[0,0]+=1; C[1,1]+=1; C[2,2]+=1
k = np.array([1,0])
dx = 0.1
dt = 0.1
order_x=2
ωS,vqS,vpS = dispersion_SellingStaggered2(k,ρ,C,dx,dt,order_x)
ωV,vqV,vpV = dispersion_Virieux(k,ρ,C,dx,dt,order_x)
assert np.allclose(ωS,ωV)
assert np.allclose(vqS,vqV)
assert np.allclose(vpS,vpV)

In [42]:
C_VTI2 = Hooke.mica[0].extract_xz().hooke
C_TTI2 = Hooke.mica[0].extract_xz().rotate_by(π/6).hooke
for C in (C_VTI2,C_TTI2): C/=np.max(C)

for ρ, C_VTI, C, k, order_x, Nx, dt, mode in [
    (1.4, C_VTI2, C_TTI2, [ 2, 3],    2, 10, 0.01, 0),
    (0.9, C_VTI2, C_TTI2, [-1, 1],    4, 10, 0.01, 0),
]:
    vdim = len(k)
    k = 2*π*np.array(k)
    X, dx = make_domain(Nx,vdim) # The grid points : offset (0,0)
    stag = staggered(dx,order_x,'Periodic')

    # Time discretization
    Nt = 1
    T = Nt*dt

    # Dispersion relation and exact solution
    ω,vq,vp = dispersion_Virieux(k,ρ,C_VTI,dx,dt,order_x)
    q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])

    q0,p0 = map(np.array,eval_Virieux(q_exact,p_exact,X,dx,0,dt))
    qf,pf = eval_Virieux(q_exact,p_exact,X,dx,T,dt)

    def af(q): return fd.as_field(q,shape=X[0].shape)
    Virieux = (None,Virieux1,Virieux2,Virieux3)[vdim] # Select adequate scheme
    scheme = Virieux(af(ρ),af(C_VTI),stag)
    q1,p1 = scheme.step(q0,p0,dt)
    assert np.allclose(qf,q1)
    assert np.allclose(pf,p1)

    # Now testing the staggered Selling scheme
    assert vdim==2
    # For VTI tensors, it should reproduce the Virieux scheme
    SellingStagH = SellingStaggered2H(ρ,C_VTI,stag,X)
    Q1,P1 = SellingStagH.Euler_p(q0,p0,dt)
    assert np.allclose(q1,Q1)
    assert np.allclose(p1,P1)

    # However, the Staggered Selling scheme also handles general elasticity
    ω,vq,vp = dispersion_SellingStaggered2(k,ρ,C,dx,dt,order_x)
    q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])
    Q0,P0 = map(np.array,eval_Virieux(q_exact,p_exact,X,dx,0,dt))
    Qf,Pf = eval_Virieux(q_exact,p_exact,X,dx,T,dt)
    SellingStagH = SellingStaggered2H(ρ,C,stag,X)
    Q1,P1 = SellingStagH.Euler_p(Q0,P0,dt)
    assert np.allclose(Qf,Q1)
    assert np.allclose(Pf,P1)

### 7.4 Comparing dispersions

The staggered Selling scheme clearly improves the dispersion relation w.r.t the original Selling scheme.
Wether this beats the Lebedev and/or correlated Selling schemes, on the pressure and/or shear waves, depends on the circumstances. A more systematic study is needed. 
(E.g. Compute approximate dispersions on all the Thomsen materials, with all the orientations.)

The mica shear wave seems to be a worst case scenario for our approach.

In [43]:
# Try changing the following parameters
dx = 1. #1.5
dt = dx/4 # Small time step, we are mostly interested on spatial dispersion
θ = π/6
C = Hooke.mica[0].extract_xz().rotate_by(θ).hooke # 
#C = Hooke.stishovite[0].extract_xz().rotate_by(θ).hooke
C /= np.max(C)

# No need to change
ρ = 1

print(f"Points per wavelength {2*π/dx=:.3f}, {order_x=}, {λ/μ=}")
print(f"Blue : Lebedev. Red : Selling.")

ak = np.linspace(-1.2,1.2)
ks = np.array(np.meshgrid(ak,ak,indexing='ij'))
def af(x): return fd.as_field(x,ks[0].shape)

ω_exact,_ = Hooke(C).waves(ks,ρ)
ω_Virieux2,_,_  = dispersion_Virieux(ks,ρ,C,dx,dt,order_x=2)
ω_Virieux4,_,_  = dispersion_Virieux(ks,ρ,C,dx,dt,order_x=4)
ω_Lebedev2,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=2)
ω_Lebedev4,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=4)

ω_Selling2,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=2,order_t=2)
ω_Selling4,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=4,order_t=2)
ω_Selling2_corr,_,_  = dispersion_SellingCorrelated(ks,af(ρ),af(C),dx,dt,order_x=2)
ω_Selling4_corr,_,_  = dispersion_SellingCorrelated(ks,af(ρ),af(C),dx,dt,order_x=4)

ω_Selling2_stag,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=2)
ω_Selling4_stag,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=4)

#ω_Selling,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=6,order_t=2) # Order 6 does not bring much benefit


fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].set_title("Shear wave dispersion"); ax[0].axis('equal')
ω0 = 0.7*ω_exact[0].mean()
ax[0].contour(*ks,ω_Lebedev2[0],levels=[ω0],colors='blue')
ax[0].contour(*ks,ω_Lebedev4[0],levels=[ω0],colors='cyan')

#ax[0].contour(*ks,ω_Selling2[0],levels=[ω0],colors='red')
#ax[0].contour(*ks,ω_Selling4[0],levels=[ω0],colors='orange')
ax[0].contour(*ks,ω_Selling2_corr[0],levels=[ω0],colors='red')
ax[0].contour(*ks,ω_Selling4_corr[0],levels=[ω0],colors='orange')

ax[0].contour(*ks,ω_Selling2_stag[0],levels=[ω0],colors='green')
ax[0].contour(*ks,ω_Selling4_stag[0],levels=[ω0],colors='olive')
ax[0].contour(*ks,ω_exact[0],levels=[ω0],colors='black')

ax[1].set_title("Pressure wave dispersion"); ax[1].axis('equal')
ω1 = 0.8*ω_exact[1].mean()
ax[1].contour(*ks,ω_Lebedev2[1],levels=[ω1],colors='blue')
ax[1].contour(*ks,ω_Lebedev4[1],levels=[ω1],colors='cyan')

#ax[1].contour(*ks,ω_Selling2[1],levels=[ω1],colors='red')
#ax[1].contour(*ks,ω_Selling4[1],levels=[ω1],colors='orange')
ax[1].contour(*ks,ω_Selling2_corr[1],levels=[ω1],colors='red')
ax[1].contour(*ks,ω_Selling4_corr[1],levels=[ω1],colors='orange')

ax[1].contour(*ks,ω_Selling2_stag[1],levels=[ω1],colors='green')
ax[1].contour(*ks,ω_Selling4_stag[1],levels=[ω1],colors='olive');
ax[1].contour(*ks,ω_exact[1],levels=[ω1],colors='black');


Points per wavelength 2*π/dx=6.283, order_x=4, λ/μ=0.4
Blue : Lebedev. Red : Selling.


### 7.5 Yet another variant

The code below implements, again, a linear program that performs a variant of Selling's decomposition with some of the considered offsets. 
We now include the extended collection of offsets $(\cos \theta,\sin \theta,0)$ for $\theta$ multiple of $\pi/6$, as well as $(0,0,1),(2,0,1),(2,2,1),(1,1,2),(4,0,1)$.
These triplets correspond to $(a,b,c)$ in the introduction of this subsection, and sign changes $(\pm a, \pm b,c)$, as well as interversions $(\pm b,\pm a,c)$ of the first two coordinates, are also allowed. 

This allows to cover all geological materials catalogued by Thomsen, and gives a bit of room in case some additional distorsion is introduced (e.g a change of coordinates in a method accounting for the topography)

<!---
Some test code.
D0 = Hooke.from_Lame(1,1).hooke
D1 = Hooke.from_Lame(1,2).hooke
D2 = Hooke.from_Lame(1,3).hooke
D3 = Hooke.from_Lame(1,4).hooke
D4 = Hooke.from_Lame(2,3).hooke
D = np.stack((D0,D1,D2,D3,D4),axis=-1)
staggered_decomp_linprogExt(D)
--->


In [44]:
_decomp_extended_precomp_vertices = np.array([[4.23205081,7.46410162,4.23205081,-3.73205081,-5.73205081,1],[1,4/3,3.30940108,-2/3,-7.60683603,2.97606774],[2.36602540,3.15470054,2.36602540,-1.57735027,-5.02072594,1.57735027],[1.5,1.15470054,1.5,-5.77350269e-01,-3,1],[2.17846097e+01,12,1,-2.88923048e+01,-6,9],[7.92820323,4,1,-5.96410162,-2,1],[1,1.92450090e-01,4/3,-9.62250449e-02,-2.66666667,1],[1,-1.28197512e-16,1,-0.5,-2,1],[2.42224319e+01,1.34074773e+01,1,-3.57411251e+01,-6.70373864,1.18149546e+01],[4.23205081,7.46410162,4.23205081,-5.73205081,-3.73205081,1],[4.78985145e+01,2.96201144e+01,2.46834287,-6.61769145e+01,-1.48100572e+01,2.07467429e+01],[7.46410162,1.49282032e+01,7.46410162,-8.21410162,-8.21410162,1],[6.36602540,3.09807621,1,-1.67320508e+01,-6,9],[4/3,1.92450090e-01,1,-2.66666667,-9.62250449e-02,1],[1,-1.28197512e-16,1,-2,-0.5,1],[1,4,7.92820323,-2,-5.96410162,1],[1.5,1.15470054,1.5,-3,-5.77350269e-01,1],[8.34298427,4.23947394,1,-2.16438643e+01,-6.47894789,1.09157915e+01],[3.30940108,4/3,1,-7.60683603,-2/3,2.97606774],[2.36602540,3.15470054,2.36602540,-5.02072594,-1.57735027,1.57735027],[1.07787504e+01,6.90264733,1.72566183,-2.84601481e+01,-1.03539710e+01,1.48052947e+01],[1,-1.28197512e-16,1,-2,-6,9],[1,-1.28197512e-16,1,-2,-2.5,2],[1.75621778e+01,-9.56217783,1,-16,-6,9],[6,2,1,-16,-6,9],[1,7.88675135e-01,2.36602540,-3.57735027,-6.30940108,4.15470054],[1.20291371,4.68609140e-01,1.20291371,-3.34304570,-3.34304570,2.87443656],[2.36602540,7.88675135e-01,1,-6.30940108,-3.57735027,4.15470054],[1,-1.28197512e-16,1,-2.5,-2,2],[4.46410162,2,1,-1.29282032e+01,-6,9],[1,1.34074773e+01,2.42224319e+01,-6.70373864,-3.57411251e+01,1.18149546e+01],[1,4.23947394,8.34298427,-6.47894789,-2.16438643e+01,1.09157915e+01],[1.72566183,6.90264733,1.07787504e+01,-1.03539710e+01,-2.84601481e+01,1.48052947e+01],[2.46834287,2.96201144e+01,4.78985145e+01,-1.48100572e+01,-6.61769145e+01,2.07467429e+01],[1,12,2.17846097e+01,-6,-2.88923048e+01,9],[1,3.09807621,6.36602540,-6,-1.67320508e+01,9],[1,2,4.46410162,-6,-1.29282032e+01,9],[1,-1.28197512e-16,1,-6,-2,9],[1,2,6,-6,-16,9],[1,-9.56217783,1.75621778e+01,-6,-16,9]])
_decomp_extended_precomp_offsets = np.array([[0,2,-2,0,0,2,-2,2,-2,1,-1,1,-1,4,-4,0,0,1,8.66025404e-01,0.5,6.12323400e-17,-0.5,-8.66025404e-01],[0,0,0,2,-2,2,2,-2,-2,1,1,-1,-1,0,0,4,-4,0,0.5,8.66025404e-01,1,8.66025404e-01,0.5],[1,1,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,0,0,0,0,0,0]])
_decomp_extended_precomp_active = np.array([[0,6,9,10,21,22],[3,6,9,10,17,22],[3,6,9,10,21,22],[0,3,9,10,21,22],[3,9,11,15,20,21],[0,3,9,11,20,21],[0,3,9,10,17,22],[0,3,9,17,19,21],[7,9,11,15,20,21],[0,7,9,11,21,22],[3,7,9,11,15,21],[0,6,7,9,21,22],[1,3,9,15,20,21],[0,1,9,11,20,21],[0,1,9,17,19,21],[0,1,9,10,17,22],[0,1,9,11,21,22],[1,7,9,15,20,21],[1,7,9,11,20,21],[1,7,9,11,21,22],[1,3,7,9,15,21],[3,5,15,17,19,21],[3,5,9,17,19,21],[3,5,9,15,19,20],[1,3,5,9,15,20],[1,3,5,9,17,22],[1,3,5,9,21,22],[1,3,5,9,20,21],[1,5,9,17,19,21],[1,3,5,15,20,21],[6,9,10,13,17,22],[3,6,9,13,17,22],[1,3,6,9,13,22],[1,6,9,10,13,22],[1,9,10,13,17,22],[1,3,9,13,17,22],[1,3,5,13,17,22],[1,5,13,17,19,21],[1,3,5,9,13,17],[1,5,9,13,17,18]])
_decomp_extended_precomp_inv = np.linalg.inv(np.moveaxis(flatten_symmetric_matrix(lp.outer_self(_decomp_extended_precomp_offsets[:,_decomp_extended_precomp_active])),0,1))
def staggered_decomp_linprogExt(D):
    shape = D.shape[2:]; assert D.shape[:2]==(3,3)
    D_flat = flatten_symmetric_matrix(D).reshape(6,-1) # Flatten additional shape
    # Find the best vertex (we are minimizing a linear form over a polyhedron)
    score = np.array([np.sum(D_flat* (np.array([1,s0*s1,1,s0,s1,1])*_decomp_extended_precomp_vertices)[:,:,None],axis=1) 
             for (s0,s1) in ((1,1),(1,-1),(-1,1),(-1,-1))])
    signs = np.argmin(np.min(score,axis=1),axis=0) 
    amin = np.argmin(np.take_along_axis(score,signs[None,None],axis=0)[0],axis=0)
    signs = np.array(((1,1),(1,-1),(-1,1),(-1,-1)))[signs].T
    ones = np.ones_like(signs[0])
    # Find the coefficients, by linear solve
    D_flat*=np.array([ones,signs[0]*signs[1],ones,signs[0],signs[1],ones])
    coefs = lp.dot_AV(np.moveaxis(_decomp_extended_precomp_inv[amin],0,-1),D_flat)
    offsets = np.moveaxis(_decomp_extended_precomp_offsets[:,_decomp_extended_precomp_active[amin]],-1,1)
    offsets[:2]*=signs[:,None]
    # Some post processing of the last offsets, so as to extract coef01, use integer coordinates...
    small = np.logical_and(offsets[2]==0,np.linalg.norm(offsets[:2],axis=0)<=1.1) # (cos θ, sin θ, 0) offsets receive special treatment
    D_small = np.sum(flatten_symmetric_matrix(lp.outer_self(offsets[:2])*np.where(small,coefs,0)),axis=1)
    coefs[small]=0; offsets[:,small]=0
    return D_small[[0,2,1]].reshape((3,*shape)),coefs[:-1].reshape((5,*shape)),offsets[:,:-1].astype(int).reshape((3,5,*shape))

In [45]:
def SellingStaggeredExt2H(ρ,C,stag,X,S=None):
    """Hamiltonian of the 2d elastic wave equation, implemented using a staggered variant of Selling's decomposition."""
    dlo,dro,dl,ar = stag.diff_left_offset,stag.diff_right_offset,stag.diff_left,stag.avg_right
    coefs_00,coefs,offsets = staggered_decomp_linprogExt(C)
    def af(x): return fd.as_field(x,X[0].shape)
    iρ = af(1/ρ)    
    iρ_10 = ar(iρ,0)
    iρ_01 = ar(iρ,1)
    assert S is None
    def PotentialEnergy(q):
        q0_10,q1_01 = q
        ϵ00_00 = dl(q0_10,0) # These terms handled similarly to Virieux
        ϵ11_00 = dl(q1_01,1) # Compute the strain tensor ϵ = (Dq+Dq^T)/2
        res = coefs_00[0]*ϵ00_00**2 + coefs_00[1]*ϵ11_00**2 +  2*coefs_00[2]*ϵ00_00*ϵ11_00
        for λ,offset in zip(coefs,np.moveaxis(offsets,-1,0)):
            if np.allclose(λ,0): continue
            e0,e1 = (offset[0],offset[2]),(offset[2],offset[1])
            at_11 = offset[2]%2==1
            ϵ = dlo(q0_10,0+at_11,e0,right=at_11) + dlo(q1_01,1-at_11,e1,right=at_11)
            # ϵ_00 = dlo(q0_10,0,e0) + dlo(q1_01,1,e1); ϵ_11 = dro(q0_10,1,e0) + dro(q1_01,0,e1); ϵ = np.where(at_11,ϵ_11,ϵ_00)
            # TODO : higher spatial accuracy by properly locating λ when at_11 is true (average with neighbors)
            res += λ * ϵ**2
        return 0.5*res
    def KineticEnergy(p):
        p0_10,p1_01 = p
        return 0.5*(iρ_10*p0_10**2 + iρ_01*p1_01**2)
    H = QuadraticHamiltonian(PotentialEnergy,KineticEnergy); H.set_spmat(np.zeros_like(X)); return H

We now validate the decomposition on the standard materials.

In [46]:
staggered_decomp_test = [Hooke.from_ThomsenElastic(tem) for name,tem in Seismic.Thomsen.ThomsenData.items()] \
    + [Hooke.mica,Hooke.stishovite,Hooke.olivine]
staggered_decomp_test = [hk[0].extract_xz().hooke for hk in staggered_decomp_test]

nθ = 100; θs = np.linspace(0,π/2,nθ)
print(f"Decomposition of {len(staggered_decomp_test)} materials, over {len(θs)} regular orientations each")

large_offset = {}
for i,m in enumerate(staggered_decomp_test):
    m = m/np.max(m) # Normalize coefficients (optional)
    for θ in θs:
        m_rot = Hooke(m).rotate_by(θ).hooke
        (a_00,b_00,c_00),coefs,offsets = staggered_decomp_linprogExt(m_rot)
        rec_00 = np.array([[a_00,c_00,0],[c_00,b_00,0],[0,0,0]])
        rec = np.sum(coefs*lp.outer_self(offsets),axis=2)
        assert np.allclose(m_rot,rec_00+rec)
        assert np.all(coefs>=-1e-6)
        assert np.all(a_00*b_00>=c_00**2)
        if np.max(np.sum(offsets**2,axis=0))>9: large_offset[i]=θ

Decomposition of 61 materials, over 100 regular orientations each


Only the biotite crystal uses the larger offsets $(4,0,1)$, for some specific orientations.

In [47]:
large_offset, list(Seismic.Thomsen.ThomsenData)[47]

({47: 1.443863290286218}, 'Biotite crystal')

Our custom implementation of the linear program can be validated by comparison with the result of a generic linear program solver.

In [48]:
def _staggered_decomp_linprogExt(m,offsets=_decomp_extended_precomp_offsets,offset_cost=None):
    # Setup and solve the linear program
    A_eq = flatten_symmetric_matrix(lp.outer_self(offsets))
    b_eq = flatten_symmetric_matrix(m)
    if offset_cost is None: c = -np.ones(A_eq.shape[1]) # Could try to adjust this objective to minimize distortion
    else: c = - np.array(list(map(offset_cost,offsets.T)))
    bounds = (0,None)
    res = linprog(c,A_eq=A_eq,b_eq=b_eq,bounds=bounds)
    #print(c); print(res.fun)
    coefs = res.x
    small = np.logical_and(offsets[2]==0,np.linalg.norm(offsets[:2],axis=0)<=1.1) # (cos θ,sin θ,0) offsets receive special treatment
    D_small = np.sum(flatten_symmetric_matrix(lp.outer_self(offsets[:2])*np.where(small,coefs,0)),axis=1)
    coefs[small]=0; offsets = offsets.copy(); offsets[:,small]=0
    select = coefs>0
    return D_small[[0,2,1]],coefs[select],offsets[:,select].astype(int)

In [49]:
_staggered_decomp_linprogExt(m_rot),staggered_decomp_linprog(m_rot)

((array([0.72628977, 1.        , 0.22119246]),
  array([0.24312635]),
  array([[0],
         [0],
         [1]])),
 ((0.7262897744825456, 1.0, 0.22119246215631758),
  array([2.43126352e-01, 0.00000000e+00, 6.93889390e-18, 0.00000000e+00]),
  array([[ 0, -2,  0, -2],
         [ 0,  0,  2,  2],
         [ 1,  1,  1,  1]])))

In [50]:
C_VTI2 = Hooke.mica[0].extract_xz().hooke
C_TTI2 = Hooke.mica[0].extract_xz().rotate_by(π/6).hooke
for C in (C_VTI2,C_TTI2): C/=np.max(C)

for ρ, C_VTI, C, k, order_x, Nx, dt, mode in [
    (1.4, C_VTI2, C_TTI2, [ 2, 3],    2, 10, 0.01, 0),
    (0.9, C_VTI2, C_TTI2, [-1, 1],    4, 11, 0.01, 0),
]:
    vdim = len(k)
    k = 2*π*np.array(k)
    X, dx = make_domain(Nx,vdim) # The grid points : offset (0,0)
    stag = staggered(dx,order_x,'Periodic')

    # Time discretization
    Nt = 1
    T = Nt*dt

    # Dispersion relation and exact solution
    ω,vq,vp = dispersion_Virieux(k,ρ,C_VTI,dx,dt,order_x)
    q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])

    q0,p0 = map(np.array,eval_Virieux(q_exact,p_exact,X,dx,0,dt))
    qf,pf = eval_Virieux(q_exact,p_exact,X,dx,T,dt)

    def af(q): return fd.as_field(q,shape=X[0].shape)
    Virieux = (None,Virieux1,Virieux2,Virieux3)[vdim] # Select adequate scheme
    scheme = Virieux(af(ρ),af(C_VTI),stag)
    q1,p1 = scheme.step(q0,p0,dt)
    assert np.allclose(qf,q1)
    assert np.allclose(pf,p1)

    # Now testing the staggered Selling scheme
    assert vdim==2
    # For VTI tensors, it should reproduce the Virieux scheme
    SellingStagH = SellingStaggeredExt2H(ρ,C_VTI,stag,X)
    Q1,P1 = SellingStagH.Euler_p(q0,p0,dt)
    assert np.allclose(q1,Q1)
    assert np.allclose(p1,P1)

    # However, the Staggered Selling scheme also handles general elasticity
    ω,vq,vp = dispersion_SellingStaggered2(k,ρ,C,dx,dt,order_x,decomp=staggered_decomp_linprogExt)
    q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])
    Q0,P0 = map(np.array,eval_Virieux(q_exact,p_exact,X,dx,0,dt))
    Qf,Pf = eval_Virieux(q_exact,p_exact,X,dx,T,dt)
    SellingStagH = SellingStaggeredExt2H(ρ,C,stag,X)
    Q1,P1 = SellingStagH.Euler_p(Q0,P0,dt)
    assert np.allclose(Qf,Q1)
    assert np.allclose(Pf,P1)

Compared to the previous staggered selling decomposition, this one appears empirically to :
- degrade the shear wave dispersion
- improve the pressure wave dispersion

This is a bit unfortunate, since the pressure wave was mostly fine already, and the approach was lacking for the shear wave. 

In applications, the pulsation is usually fixed. Since the pressure wave is faster, it has a longer wavelength. Therefore the bottleneck in terms of dispersion is really the shear wave.

In [51]:
# Try changing the following parameters
dx = 1. #1.5
dt = dx/4 # Small time step, we are mostly interested on spatial dispersion
θ = 0.7 #π/6
C = Hooke.mica[0].extract_xz().rotate_by(θ).hooke 
#C = Hooke.stishovite[0].extract_xz().rotate_by(θ).hooke
C /= np.max(C)

# No need to change
ρ = 1

print(f"Points per wavelength {2*π/dx=:.3f}, {order_x=}, {λ/μ=}")
print(f"Blue : Selling stag extended. Red : Selling correlated. Green : Selling stag")

ak = np.linspace(-1.2,1.2)
ks = np.array(np.meshgrid(ak,ak,indexing='ij'))
def af(x): return fd.as_field(x,ks[0].shape)

ω_exact,_ = Hooke(C).waves(ks,ρ)
#ω_Virieux2,_,_  = dispersion_Virieux(ks,ρ,C,dx,dt,order_x=2)
#ω_Virieux4,_,_  = dispersion_Virieux(ks,ρ,C,dx,dt,order_x=4)
ω_Lebedev2,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=2)
ω_Lebedev4,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=4)

#ω_Selling2,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=2,order_t=2)
#ω_Selling4,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=4,order_t=2)
ω_Selling2_corr,_,_  = dispersion_SellingCorrelated(ks,af(ρ),af(C),dx,dt,order_x=2)
ω_Selling4_corr,_,_  = dispersion_SellingCorrelated(ks,af(ρ),af(C),dx,dt,order_x=4)

ω_Selling2_stag,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=2)
ω_Selling4_stag,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=4)

ω_Selling2_stagExt,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=2,decomp=staggered_decomp_linprogExt)
ω_Selling4_stagExt,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=4,decomp=staggered_decomp_linprogExt)

#ω_Selling,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=6,order_t=2) # Order 6 does not bring much benefit


fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].set_title("Shear wave dispersion"); ax[0].axis('equal')
ω0 = 0.7*ω_exact[0].mean()
#ax[0].contour(*ks,ω_Lebedev2[0],levels=[ω0],colors='blue')
#ax[0].contour(*ks,ω_Lebedev4[0],levels=[ω0],colors='cyan')
ax[0].contour(*ks,ω_Selling2_stagExt[0],levels=[ω0],colors='blue')
ax[0].contour(*ks,ω_Selling4_stagExt[0],levels=[ω0],colors='cyan')

#ax[0].contour(*ks,ω_Selling2[0],levels=[ω0],colors='red')
#ax[0].contour(*ks,ω_Selling4[0],levels=[ω0],colors='orange')
ax[0].contour(*ks,ω_Selling2_corr[0],levels=[ω0],colors='red')
ax[0].contour(*ks,ω_Selling4_corr[0],levels=[ω0],colors='orange')

ax[0].contour(*ks,ω_Selling2_stag[0],levels=[ω0],colors='green')
ax[0].contour(*ks,ω_Selling4_stag[0],levels=[ω0],colors='olive')
ax[0].contour(*ks,ω_exact[0],levels=[ω0],colors='black')

ax[1].set_title("Pressure wave dispersion"); ax[1].axis('equal')
ω1 = 0.8*ω_exact[1].mean()
#ax[1].contour(*ks,ω_Lebedev2[1],levels=[ω1],colors='blue')
#ax[1].contour(*ks,ω_Lebedev4[1],levels=[ω1],colors='cyan')
ax[1].contour(*ks,ω_Selling2_stagExt[1],levels=[ω1],colors='blue')
ax[1].contour(*ks,ω_Selling4_stagExt[1],levels=[ω1],colors='cyan')

#ax[1].contour(*ks,ω_Selling2[1],levels=[ω1],colors='red')
#ax[1].contour(*ks,ω_Selling4[1],levels=[ω1],colors='orange')
ax[1].contour(*ks,ω_Selling2_corr[1],levels=[ω1],colors='red')
ax[1].contour(*ks,ω_Selling4_corr[1],levels=[ω1],colors='orange')

ax[1].contour(*ks,ω_Selling2_stag[1],levels=[ω1],colors='green')
ax[1].contour(*ks,ω_Selling4_stag[1],levels=[ω1],colors='olive');
ax[1].contour(*ks,ω_exact[1],levels=[ω1],colors='black');


Points per wavelength 2*π/dx=6.283, order_x=4, λ/μ=0.4
Blue : Selling stag extended. Red : Selling correlated. Green : Selling stag


The extended staggered Selling decomposition considered here appears to favor the offset 
$$
    \begin{pmatrix}
        1 & 2 \\
        2 & 1
    \end{pmatrix},
$$
whereone the previous one favored 
$$
    \begin{pmatrix}
        2 & 1 \\
        1 & 2
    \end{pmatrix}.
$$
The larger off-diagonal terms of the former lead to stronger cross-derivatives terms, which may explain the degradation of dispersion (despite the use of a staggered grid). 
We could adjust for this behavior by introducing weights in the linear program objective functional, so as to discourage the use of matrix offsets with large off-diagonal components. This does not work so well though, in practice, apparently.

Another empirical observation is that, curiously, the extended staggered Selling decomposition often does not use the offset 
$$
    \begin{pmatrix}
        0 & 1 \\
        1 & 0
    \end{pmatrix},
$$
despite its smallness.

In [52]:
staggered_decomp_linprog(C),staggered_decomp_linprogExt(C),

(((0.17492306746144626, 0.23156171797975833, 0.05029956505001132),
  array([0.3522868 , 0.07276302, 0.        , 0.13350621]),
  array([[0, 2, 0, 2],
         [0, 0, 2, 2],
         [1, 1, 1, 1]])),
 (array([0.52720986, 0.58384852, 0.40258636]),
  array([0.07276302, 0.01607728, 0.11742893, 0.        , 0.        ]),
  array([[2, 2, 1, 0, 0],
         [0, 2, 1, 0, 0],
         [1, 1, 2, 0, 0]])))

In [53]:
def offset_cost(offset): 
    return 1/(1+2*offset[2]**2)
#    return float(tuple(offset.astype(int)) in [(0,0,1),(2,0,1),(0,2,1),(2,2,1)])
#    return 0.+(offset[2]==1) 
ϕs = np.linspace(0,π,120)
myoffsets = np.concatenate((_decomp_extended_precomp_offsets[:,:-6], [np.cos(ϕs),np.sin(ϕs),0*ϕs]),axis=1)
def mydecomp(C): return _staggered_decomp_linprogExt(C,offsets=myoffsets,offset_cost=offset_cost)

mydecomp(C)

(array([0.52720986, 0.58384852, 0.40258636]),
 array([0.07276302, 0.01607728, 0.11742893]),
 array([[2, 2, 1],
        [0, 2, 1],
        [1, 1, 2]]))

In [54]:
# Try changing the following parameters
dx = 1.2 #1.5
dt = dx/4 # Small time step, we are mostly interested on spatial dispersion
θ = 0.4 #π/6
C = Hooke.mica[0].extract_xz().rotate_by(θ).hooke 
#C = Hooke.stishovite[0].extract_xz().rotate_by(θ).hooke
C /= np.max(C)

# No need to change
ρ = 1

print(f"Points per wavelength {2*π/dx=:.3f}, {order_x=}, {λ/μ=}")
print(f"Blue : Selling stag extended. Red : Selling correlated. Green : Selling stag")

ak = np.linspace(-1.2,1.2)
ks = np.array(np.meshgrid(ak,ak,indexing='ij'))
def af(x): return fd.as_field(x,ks[0].shape)

ω_exact,_ = Hooke(C).waves(ks,ρ)
#ω_Virieux2,_,_  = dispersion_Virieux(ks,ρ,C,dx,dt,order_x=2)
#ω_Virieux4,_,_  = dispersion_Virieux(ks,ρ,C,dx,dt,order_x=4)
ω_Lebedev2,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=2)
ω_Lebedev4,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=4)

#ω_Selling2,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=2,order_t=2)
#ω_Selling4,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=4,order_t=2)
ω_Selling2_corr,_,_  = dispersion_SellingCorrelated(ks,af(ρ),af(C),dx,dt,order_x=2)
ω_Selling4_corr,_,_  = dispersion_SellingCorrelated(ks,af(ρ),af(C),dx,dt,order_x=4)

ω_Selling2_stag,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=2)
ω_Selling4_stag,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=4)

ω_Selling2_stagExt,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=2,decomp=staggered_decomp_linprogExt)
ω_Selling4_stagExt,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=4,decomp=staggered_decomp_linprogExt)

ω_Selling2_mystagExt,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=2,decomp=mydecomp)
ω_Selling4_mystagExt,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=4,decomp=mydecomp)

#ω_Selling,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=6,order_t=2) # Order 6 does not bring much benefit


fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].set_title("Shear wave dispersion"); ax[0].axis('equal')
ω0 = 0.7*ω_exact[0].mean()
#ax[0].contour(*ks,ω_Lebedev2[0],levels=[ω0],colors='blue')
#ax[0].contour(*ks,ω_Lebedev4[0],levels=[ω0],colors='cyan')
ax[0].contour(*ks,ω_Selling2_stagExt[0],levels=[ω0],colors='blue')
ax[0].contour(*ks,ω_Selling4_stagExt[0],levels=[ω0],colors='cyan')

#ax[0].contour(*ks,ω_Selling2[0],levels=[ω0],colors='red')
#ax[0].contour(*ks,ω_Selling4[0],levels=[ω0],colors='orange')
#ax[0].contour(*ks,ω_Selling2_corr[0],levels=[ω0],colors='red')
#ax[0].contour(*ks,ω_Selling4_corr[0],levels=[ω0],colors='orange')
ax[0].contour(*ks,ω_Selling2_mystagExt[0],levels=[ω0],colors='red')
ax[0].contour(*ks,ω_Selling4_mystagExt[0],levels=[ω0],colors='orange')

ax[0].contour(*ks,ω_Selling2_stag[0],levels=[ω0],colors='green')
ax[0].contour(*ks,ω_Selling4_stag[0],levels=[ω0],colors='olive')
ax[0].contour(*ks,ω_exact[0],levels=[ω0],colors='black')

ax[1].set_title("Pressure wave dispersion"); ax[1].axis('equal')
ω1 = 0.8*ω_exact[1].mean()
#ax[1].contour(*ks,ω_Lebedev2[1],levels=[ω1],colors='blue')
#ax[1].contour(*ks,ω_Lebedev4[1],levels=[ω1],colors='cyan')
ax[1].contour(*ks,ω_Selling2_stagExt[1],levels=[ω1],colors='blue')
ax[1].contour(*ks,ω_Selling4_stagExt[1],levels=[ω1],colors='cyan')

#ax[1].contour(*ks,ω_Selling2[1],levels=[ω1],colors='red')
#ax[1].contour(*ks,ω_Selling4[1],levels=[ω1],colors='orange')
#ax[1].contour(*ks,ω_Selling2_corr[1],levels=[ω1],colors='red')
#ax[1].contour(*ks,ω_Selling4_corr[1],levels=[ω1],colors='orange')
ax[1].contour(*ks,ω_Selling2_mystagExt[1],levels=[ω1],colors='red')
ax[1].contour(*ks,ω_Selling4_mystagExt[1],levels=[ω1],colors='orange')

ax[1].contour(*ks,ω_Selling2_stag[1],levels=[ω1],colors='green')
ax[1].contour(*ks,ω_Selling4_stag[1],levels=[ω1],colors='olive');
ax[1].contour(*ks,ω_exact[1],levels=[ω1],colors='black');

Points per wavelength 2*π/dx=5.236, order_x=4, λ/μ=0.4
Blue : Selling stag extended. Red : Selling correlated. Green : Selling stag


## 8. Energy conservation

*This section is an (unconvincing) draft. Evaluating the Selling energy on the Lebedev scheme solution is somewhat unfair. Some other way to illustrate the issue of grid decoupling ?*

The proposed Selling scheme conserves a perturbed energy. As a result, the true system energy is conserved up to a multiplicative constant (which is close to one and if the CFL is satisfied strictly).

The Virieux and Lebedev schemes can also be regarded as (Euler) symplectic schemes, based on a suitable potential energy. 
- The Virieux energy is rather simple and natural, and one can prove that it controls the isotropic energy
$$
\| \partial^0q^0\|^2 + \|\partial^1 q^1\|^2+ \|\partial^1 q^0+\partial^0 q^1\|^2.
$$
As a result, one can obtain e.g discrete Korn estimates.
- The Lebedev energy couples the subgrids in a way that makes it difficult to prove local estimates like the above, or simply the scheme convergence.

In the following (admittedly unphysical) example where the Hooke tensors are chosen at random, the we observe that : 

- The (Selling) energy of the Lebedev solution oscillates rather widly. 
- (**This point is unclear after all**) For non-VTI hooke tensors, the Lebedev energy usually overestimates the true energy. In contrast it appears to oscillate centered on the true energy in the VTI case (In that case, Lebedev reduces to two independent Virieux schemes, which are symplectic for reasonnable energy).
- Oscillations are larger for the 4th order variant of the Lebedev scheme.

In [55]:
Nt = 2000
Nx = 10
vdim = 2
order_x = 2
bc = 'Periodic'
cfl = 0.7
plt.figure(figsize=[12,6])

np.random.seed(42)
X, dx = make_domain(Nx,vdim) # The grid points : offset (0,0)
stag = staggered(dx,order_x,bc)

C = MakeRandomTensor(3,(Nx,Nx),0.5); C/=np.max(C,axis=(0,1)) # Random Hooke elasticity tensors
ρ = np.random.rand(Nx,Nx)+0.5

q0_Lebedev = np.random.rand(2,2,Nx,Nx)
p0_Lebedev = np.random.rand(2,2,Nx,Nx)

q0_Selling = np.array((q0_Lebedev[0,0],q0_Lebedev[1,1]))
p0_Selling = np.array((p0_Lebedev[0,0],p0_Lebedev[1,1]))

q_Lebedev,p_Lebedev,q_Selling,p_Selling = copy.deepcopy((q0_Lebedev,p0_Lebedev,q0_Selling,p0_Selling))

for VTI in (False,True):
    if VTI: C[0,2]=C[1,2]=C[2,0]=C[2,1]=0

    scheme_Lebedev = Lebedev2(ρ,C,stag).step
    scheme_Virieux = Virieux2(ρ,C,stag).step
    WaveH = AnisotropicWave.ElasticHamiltonian_Sparse(ρ[None,None],C,dx,order_x,bc=bc)
    scheme_Selling = WaveH.Verlet_p

    energy_Selling  = lambda q,p : WaveH.H(q,p)
    energy_Selling2 = lambda q,p : WaveH.H_p(q,p,dt,order=2)
    energy_Lebedev  = lambda q,p : 0.5*(WaveH.H(np.array((q[0][0],q[1][1])),np.array((p[0][0],p[1][1]))) \
                                  + WaveH.H(np.array((q[0][1],q[1][0])),np.array((p[0][1],p[1][0]))))

    hist_Selling  = []
    hist_Selling2 = []
    hist_Lebedev  = []

    dt = cfl*WaveH.dt_max()
    for i in range(Nt):
        q_Lebedev,p_Lebedev = scheme_Lebedev(q_Lebedev,p_Lebedev,dt)    
        q_Selling,p_Selling = scheme_Selling(q_Selling,p_Selling,dt)

        hist_Lebedev.append(energy_Lebedev(q_Lebedev,p_Lebedev))
        hist_Selling.append(energy_Selling(q_Selling,p_Selling))
        hist_Selling2.append(energy_Selling2(q_Selling,p_Selling))

    plt.subplot(1,2,1+VTI)
    plt.title(f"Numerical energy (non-)conservation. {order_x=}, {VTI=}")
    plt.plot(hist_Selling,label="Selling energy")
    plt.plot(hist_Selling2,label="Selling invariant energy")
    plt.plot(hist_Lebedev,label="Lebedev energy")
    plt.legend();

## 9. Grid decoupling in the Lebedev scheme

Let us verify that the Lebedev scheme features two independent subgrids in the case of VTI tensors, each of them equivalent to a Virieux scheme.

In [56]:
# Choose the problem parameters
np.random.seed(42)
q0_Lebedev = np.random.rand(2,2,Nx,Nx)
p0_Lebedev = np.random.rand(2,2,Nx,Nx)

def af(q): return fd.as_field(q,shape=X[0].shape)
#ρ = af(1)
ρ = np.random.rand(Nx,Nx)+0.5

# We need C to have a VTI structure
#C = af(Hooke.from_Lame(1,1).hooke)
C = MakeRandomTensor(3,(Nx,Nx),0.5); C/=np.max(C,axis=(0,1)); C[0,2]=C[1,2]=C[2,0]=C[2,1]=0 # Random VTI elasticity tensors

# The Virieux unknowns are extracted from the Lebedev unknowns in a specific staggered manner
def extract_a(q): return (q[0][0],q[1][1])
q0_Virieux_a = extract_a(q0_Lebedev)
p0_Virieux_a = extract_a(p0_Lebedev)
def extract_b(q): return np.roll(q[1][0],-1,axis=0),np.roll(q[0][1],-1,axis=1)
q0_Virieux_b = extract_b(q0_Lebedev)
p0_Virieux_b = extract_b(p0_Lebedev)

scheme_Lebedev = Lebedev2(ρ,C,stag)
scheme_Virieux_a = Virieux2(ρ,C,stag)

# We must carefully shift coefficients of the second Virieux scheme
scheme_Virieux_b = copy.copy(scheme_Virieux_a)
scheme_Virieux_b.iρ_10 = np.roll(scheme_Virieux_a.iρ_01,-1,axis=0)
scheme_Virieux_b.iρ_01 = np.roll(scheme_Virieux_a.iρ_10,-1,axis=1)
scheme_Virieux_b.C00_00 = scheme_Lebedev.C_11[0,0]
scheme_Virieux_b.C01_00 = scheme_Lebedev.C_11[0,1]
scheme_Virieux_b.C11_00 = scheme_Lebedev.C_11[1,1]
scheme_Virieux_b.C22_11 = np.roll(scheme_Lebedev.C_00[2,2],(-1,-1),axis=(0,1))

q1_Lebedev,p1_Lebedev = scheme_Lebedev.step(q0_Lebedev,p0_Lebedev,dt)
q1_Virieux_a,p1_Virieux_a = scheme_Virieux_a.step(q0_Virieux_a,p0_Virieux_a,dt)
q1_Virieux_b,p1_Virieux_b = scheme_Virieux_b.step(q0_Virieux_b,p0_Virieux_b,dt)
assert np.allclose(q1_Virieux_a,extract_a(q1_Lebedev))
assert np.allclose(q1_Virieux_b,extract_b(q1_Lebedev))

## 10. Acoustic scheme

We introduced a numerical scheme for the acoustic wave equation, based on Selling's decomposition, in a [previous notebook](HighOrderWaves).
Let us compare it to a standard non-monotone approach.

### 10.1 Centered scheme

We implement a basic scheme for the hamiltonian of the acoustic wave equation, using standard finite differences along the coordinate axes.
Note that we need to use centered finite differences for the off diagonal terms : a staggered grid would not be usable here, since there is only one field component.

In [57]:
def AcousticCenteredH(ρ,D,X,dx,order_x=2,bc='Periodic'):
    padding = {'Periodic':None,'Dirichlet':0}[bc]
    vdim = len(D)
    def PotentialEnergy(q): # q is stored at grid points
        e = np.eye(vdim).astype(int)
        dq = fd.DiffCentered(q,e,dx,order_x,padding=padding) # Centered finite differences, periodic b.c
        dq2 = np.sum(fd.DiffEll(q,e,dx,order_x,padding=padding)**2,axis=0) # Upwind and downwind finite differences, squared and summed.
        diag = sum(D[i,i]*dq2[i] for i in range(vdim)) # Sum of squares for the diagonal
        offdiag = sum(2*D[i,j]*dq[i]*dq[j] for i in range(vdim) for j in range(i)) if vdim>=2 else 0
        return 0.5*(diag+offdiag)
    def KineticEnergy(p):
        return 0.5*p**2/ρ
    H = QuadraticHamiltonian(PotentialEnergy,KineticEnergy); H.set_spmat(np.zeros_like(X[0])); return H

### 10.2 Criss-cross scheme

The criss cross-scheme is the closest to a staggered grid scheme in the acoustic setting. 
The second order variant can be reformulated in terms of diagonal finite differences, hence the name.

In comparison with the centered scheme the criss-cross scheme 
- worse for axis-aligned derivatives, corresponding to diagonal terms in the matrix $D$.
- better for cross derivatives, corresponding to off diagonal terms in the matrix $D$.

In addition, and similarly to the Lebedev scheme, the criss-cross scheme suffers from grid decoupling when the matrix D is isotropic (or when the anisotropy is along the diagonals).

In [58]:
def AcousticCrissCrossH(ρ,D,X,dx,stag):
    dl,dr,al,ar = stag.diff_left,stag.diff_right,stag.avg_left,stag.avg_right
    vdim = len(D)
    iρ_1d = ar(1/ρ,tuple(range(vdim))) # 1d means At position 1, 11, or 111, in dimension d = 1, 2 or 3
    def PotentialEnergy(q_1d): 
        # Compute a centered gradient using centered averages and differences
        dq_0d = ad.array([al(dl(q_1d,i),tuple(range(i))+tuple(range(i+1,vdim))) for i in range(vdim)])
        return 0.5 * lp.dot_VAV(dq_0d,D,dq_0d)
    def KineticEnergy(p_1d):
        return 0.5 * p_1d**2 * iρ_1d
    H = QuadraticHamiltonian(PotentialEnergy,KineticEnergy); H.set_spmat(np.zeros_like(X[0])); return H

### 10.3 Dispersion

We compute the numerical dispersion of the previous scheme, and use it to cross validate the implementation.
Recall that the dispersion relation in the continuous setting reads 
\begin{align*}
    -\ri \omega w &= v/\rho &
    -\ri \omega v &= \<\kw,D \kw\> w.
\end{align*}
where $q = w \be(\<\kw,x\>-\omega t)$ and $p = v \be(\<\kw,x\>-\omega t)$, for some scalars $v,w$, pulsation $\omega$, and wavenumber $\kw$.
Recall that $\be(s) := \exp(\ri s)$ denotes the complex exponential, and that the acoustic wave equation reads $\dot q=p/\rho$, and $\dot p = \mathrm{div}(D \nabla q)$.

In [59]:
def _dispersion_DiffCentered(s,h,order_x):
    """Fourier transforme of the centered finite difference approximation of first derivative."""
    sh = s*h
    if order_x==2: return np.sin(sh)/h
    if order_x==4: return ((4/3)*np.sin(sh)-(1/6)*np.sin(2*sh))/h
    if order_x==6: return ((3/2)*np.sin(sh)-(3/10)*np.sin(2*sh)+(1/30)*np.sin(3*sh))/h

def _dispersion_Diff2(s,h,order_x): 
    """Fourier transform of finite difference approximation of second derivative."""
    sh = s*h
    if order_x==2: return 4*np.sin(sh/2)**2/h**2
    if order_x==4: return (np.cos(2*sh)-16*np.cos(sh)+15)/(6*h**2)
    if order_x==6: return (49/18-3*np.cos(sh)+3/10*np.cos(2*sh)-1/45*np.cos(3*sh))/h**2

def _dispersion_AcousticT2(ρ,Iω2,dt):
    """Helper implementing the dispersion for a second order accurate discretization in time of a scalar equation"""
    Iω = np.sqrt(Iω2)
    ω = _dispersion_Iinv(Iω,dt)
    vq = 1. # Amplitude of position q
    vp = -Iω*ρ*vq # Amplitude of impulsion p
    return ω,vq,vp

def dispersion_AcousticCentered(k,ρ,D,dx,dt,order_x=2):
    """Dispersion relation for the centered non-monotone finite differences approximation of the acoustic wave equation."""
    dk  = _dispersion_DiffCentered(k,dx,order_x)
    dk2 = _dispersion_Diff2(k,dx,order_x)
    vdim=len(k)
    diag = sum(D[i,i]*dk2[i] for i in range(vdim)) 
    offdiag = sum(2*D[i,j]*dk[i]*dk[j] for i in range(vdim) for j in range(i))
    Iω2 = (diag+offdiag)/ρ
    return _dispersion_AcousticT2(ρ,Iω2,dt)

def dispersion_AcousticCrissCross(k,ρ,D,dx,dt,order_x=2):
    """Dispersion relation for the criss-cross non-monotone finite differences approximation of the acoustic wave equation."""
    vdim = len(k)
    dk = np.array([np.prod([_dispersion_DiffStaggered(k[j],dx,order_x) if i==j else _dispersion_AvgStaggered(k[j],dx,order_x)
                        for j in range(vdim)],axis=0) for i in range(vdim)])
    Iω2 = lp.dot_VAV(dk,D/ρ,dk)
    return _dispersion_AcousticT2(ρ,Iω2,dt)

Let us numerically validate the dispersion relation of this scheme.

<!---
# Domain
Nx = 10
vdim = 2
X,dx = make_domain(Nx,vdim)
order_x = 2

# Model parameters
k = 2*π*np.array((2,1))
ρ = 1.5
D = Riemann.from_diagonal((1,4**2)).rotate_by(np.pi/6).m
#k = 2*π*np.array([1]); D=np.array([[1]])
#D = np.eye(2)
#D = np.array([[1,0.1],[0.1,1]])

# Generate solution and Hamiltonian
ω,vq,vp = dispersion_Acoustic(k,ρ,D,dx,dt,order_x)
q_exact,p_exact = mk_planewave_e(k,ω,vq,vp)
WaveH = AcousticH(ρ,D,X,dx,order_x)

# Time discretization
dt = 0.1 
Nt = 1
T = Nt*dt

# Run solver
q0 = q_exact(dt/2,X); p0 = p_exact(0,X)
qf = q_exact(T+dt/2,X); pf = p_exact(T,X)

q1,p1 = WaveH.Euler_p(q0,p0,dt)

assert np.allclose(q1,qf)


ρ = 1
D = np.eye(2)
#D = np.array([[0,1.],[1.,0]])
k = 2*π*np.array([1,1])
q0 = np.exp(1j*(k[0]*X[0]+k[1]*X[1]))
WaveH = AcousticH(ρ,D,X,dx,order_x)
ratio_ = WaveH._DqH(q0)/q0
ratio = np.real(ratio_[0,0])
assert np.allclose(ratio_,ratio)
ratio #,_dispersion_DiffCentered(
--->

<!---
k0 = [1]
Nx = 10
order_x = 2
D = np.array([[1.2]])
ρ = 1.
k = 2*π*np.array(k0) # Wavenumber must be an integer multiple of 2π for periodicity
dt=0.1
vdim = len(k)
X,dx = make_domain(Nx,vdim)
Nt = 1
T = Nt*dt
print(f"{vdim=}, {order_x=}. ",end="")

ω,vq,vp = dispersion_AcousticCrissCross(k,ρ,D,dx,dt,order_x)
q_exact,p_exact = mk_planewave_e(k,ω,vq,vp)
def af(x): return fd.as_field(x,X[0].shape)
stag = staggered(dx,order_x,'Periodic')
WaveH = AcousticCrissCrossH(af(ρ),af(D),X,dx,stag)
X_1d = X+dx/2
q0 = q_exact(dt/2,  X_1d); p0 = p_exact(0,X_1d)
qf = q_exact(T+dt/2,X_1d); pf = p_exact(T,X_1d)
q1,p1 = WaveH.Euler_p(q0,p0,dt)
assert np.allclose(q1,qf)
assert np.allclose(p1,pf)

WaveH._DqH(q0)/q0, _dispersion_DiffStaggered(k[0],dx,order_x)**2
--->

In [60]:
D1 = np.array([[1.2]])
D2 = Riemann.from_diagonal((1,4**2)).rotate_by(np.pi/6).m
#D2 = np.eye(2)
D3 = Riemann.from_diagonal((0.6,0.8,1.3)).rotate_by(np.pi/8,(1,2,3)).m

for Nx,order_x,k0,ρ,dt,D in [
    (13,2,(1,),0.8,0.1,D1),
    (13,6,(1,),0.8,0.1,D1),
    (12,2,(1,2),1.5,0.05,D2),
    (9,4,(1,-2),1.,0.03,D2),
    (7,6,(1,1),2.,0.03,D2),
    (10,2,(1,2,3),2.,0.02,D3),
]:
    # Domain parameters
    k = 2*π*np.array(k0) # Wavenumber must be an integer multiple of 2π for periodicity
    vdim = len(k)
    X,dx = make_domain(Nx,vdim)
    Nt = 1
    T = Nt*dt
    print(f"{vdim=}, {order_x=}. ",end="")

    # ----- Centered scheme -----
    # Generate solution and Hamiltonian
    ω,vq,vp = dispersion_AcousticCentered(k,ρ,D,dx,dt,order_x)
    q_exact,p_exact = mk_planewave_e(k,ω,vq,vp)
    WaveH = AcousticCenteredH(ρ,D,X,dx,order_x)

    # Run propagation and check results
    q0 = q_exact(dt/2,X); p0 = p_exact(0,X)
    qf = q_exact(T+dt/2,X); pf = p_exact(T,X)
    q1,p1 = WaveH.Euler_p(q0,p0,dt)
    assert np.allclose(q1,qf)
    assert np.allclose(p1,pf)
    print("Centered passed. ",end="")

    if order_x>4: print(); continue
    # ----- Criss-cross scheme -----
    ω,vq,vp = dispersion_AcousticCrissCross(k,ρ,D,dx,dt,order_x)
    q_exact,p_exact = mk_planewave_e(k,ω,vq,vp)
    def af(x): return fd.as_field(x,X[0].shape,conditional=False)
    stag = staggered(dx,order_x,'Periodic')
    WaveH = AcousticCrissCrossH(af(ρ),af(D),X,dx,stag)

    # Run propagation and check results
    X_1d = X+dx/2
    q0 = q_exact(dt/2,  X_1d); p0 = p_exact(0,X_1d)
    qf = q_exact(T+dt/2,X_1d); pf = p_exact(T,X_1d)
    q1,p1 = WaveH.Euler_p(q0,p0,dt)
    assert np.allclose(q1,qf)
    assert np.allclose(p1,pf)
    print("Criss-cross passed.")

vdim=1, order_x=2. Centered passed. Criss-cross passed.
vdim=1, order_x=6. Centered passed. 
vdim=2, order_x=2. Centered passed. Criss-cross passed.
vdim=2, order_x=4. Centered passed. Criss-cross passed.
vdim=2, order_x=6. Centered passed. 
vdim=3, order_x=2. Centered passed. Criss-cross passed.


We now compare the dispersion of this standard scheme, with the Selling based scheme introduced in [another notebook](HighOrderWaves.ipynb).
Note that the Selling based scheme is originally intended to provide improved stability, since it is a monotone scheme. 

However, it turns out that it strongly improves dispersion too. One of the key reasons seems to be that it does not rely on cross derivatives. In the example below, the fourth order Selling scheme will apparently do with 2 points per wavelength for anisotropy in a "generic orientation", and three points per wavelength for axis-aligned anisotropy (which strangely turns out to be *less* favourable than some more random orientation).

In [61]:
θ = 0.5 #π/8
κ = 3
D = Riemann.from_diagonal((1,κ**2)).rotate_by(θ).m; D/=np.max(D)
dx = 2
dt = 0.1
ak = np.linspace(-1.2,1.2)
ks = np.array(np.meshgrid(ak,ak,indexing='ij'))
ρ = 1
ω_exact = np.sqrt(lp.dot_VAV(ks,D[:,:,None,None]/ρ,ks))
ω_centered2,_,_ = dispersion_AcousticCentered(ks,ρ,D,dx,dt,2)
ω_centered4,_,_ = dispersion_AcousticCentered(ks,ρ,D,dx,dt,4)
ω_centered6,_,_ = dispersion_AcousticCentered(ks,ρ,D,dx,dt,6)
def af(x):return fd.as_field(x,ks.shape[1:])
ω_crisscross2,_,_ = dispersion_AcousticCrissCross(ks,af(ρ),af(D),dx,dt,2)
ω_crisscross4,_,_ = dispersion_AcousticCrissCross(ks,af(ρ),af(D),dx,dt,4)

ω_selling2,_ = dispersion_a(ks,af(ρ),af(D),dx,dt,2)
ω_selling4,_ = dispersion_a(ks,af(ρ),af(D),dx,dt,4)

ωmin,ωmax = np.sqrt(np.linalg.eigvalsh(D/ρ)) # Smallest and largest pulsation when |k|=1
ωref = ωmin
plt.contour(*ks,ω_exact,levels=[ωref],colors='black')
plt.contour(*ks,ω_centered2,levels=[ωref],colors='blue')
plt.contour(*ks,ω_centered4,levels=[ωref],colors='cyan')
plt.contour(*ks,ω_crisscross2,levels=[ωref],colors='green')
plt.contour(*ks,ω_crisscross4,levels=[ωref],colors='olive')
plt.contour(*ks,ω_selling2,levels=[ωref],colors='red')
plt.contour(*ks,ω_selling4,levels=[ωref],colors='orange')

plt.axis('equal'); 

print(f"Number of points per wavelength {2*π/dx}")

Number of points per wavelength 3.141592653589793


## 11. Image and animation exports

We reproduce some of the previous figures, with nice parameters, and animations.

In [62]:
if savefig.dirName is None: 
    raise ad.DeliberateNotebookError("Main notebook ends here.")

In [87]:
def to_filename(str): 
    for key,val in (', ','-'),(' ','-'),('κ','kappa'): str = str.replace(key,val)
    return str

### 11.1 Acoustic

In [97]:
def frame_acoustic(κ,θ,dx,order_x,ks,title):
    plt.clf()
    D = Riemann.from_diagonal((κ**-2,1)).rotate_by(θ).m
    def af(x):return fd.as_field(x,ks.shape[1:])
    ωref = κ**-1; dt = 0.01
    ω_exact = np.sqrt(lp.dot_VAV(ks,D[:,:,None,None]/ρ,ks))
    ω_centered,_,_ = dispersion_AcousticCentered(ks,ρ,D,dx,dt,order_x)
    ω_crisscross,_,_ = dispersion_AcousticCrissCross(ks,af(ρ),af(D),dx,dt,order_x)
    ω_selling,_ = dispersion_a(ks,af(ρ),af(D),dx,dt,order_x)
    plt.contour(*ks,ω_exact,levels=[ωref],colors='black')
    plt.contour(*ks,ω_centered,levels=[ωref],colors='blue')
    plt.contour(*ks,ω_crisscross,levels=[ωref],colors='green')
    plt.contour(*ks,ω_selling,levels=[ωref],colors='red')
    plt.title(title)
    plt.legend(loc='lower right',handles=[Patch(color=color,label=label) for color,label in 
                    (('black','exact'),('blue','centered'),('green','crisscross'),('red','selling'))])
def mk_ks(kmax): 
    ak = np.linspace(-kmax,kmax)
    return np.array(np.meshgrid(ak,ak,indexing='ij'))

In [98]:
fig = plt.figure(figsize=[6,6]); plt.axis('equal')
ppw = 4; κ=3; θ=π/6; order_x=2; kmax = 1.2 # parameters
dx = 2*π/ppw; ks = mk_ks(kmax); title = f"Acoustic dispersions, {κ=}, {order_x=}, {ppw=}"
frame_acoustic(κ,θ,dx,order_x,ks,title)
savefig(fig,to_filename(title))

In [99]:
fig = plt.figure(figsize=[6,6]); plt.axis('equal')
ppw = 4; κ=3; θ=π/6; order_x=4; kmax = 1.2 # parameters
dx = 2*π/ppw; ks = mk_ks(kmax); title = f"Acoustic dispersions, {κ=}, {order_x=}, {ppw=}"
frame_acoustic(κ,θ,dx,order_x,ks,title)
savefig(fig,to_filename(title))

In [107]:
fig = plt.figure(figsize=[6,6]); plt.axis('equal')
ppw = 4; κ=3; θs=np.linspace(0,π/2,30); order_x=2; kmax = 1.4 # parameters
dx = 2*π/ppw; ks = mk_ks(kmax); title = f"Acoustic dispersions, {κ=}, {order_x=}, {ppw=}"
anim = animation.FuncAnimation(fig, lambda i:frame_acoustic(κ,θs[i],dx,order_x,ks,title), frames=len(θs), repeat=False)
anim.save(savefig.dirName+to_filename(title)+'.mp4'); anim 

In [108]:
fig = plt.figure(figsize=[6,6]); plt.axis('equal')
ppw = 4; κ=3; θs=np.linspace(0,π/2,30); order_x=4; kmax = 1.2 # parameters
dx = 2*π/ppw; ks = mk_ks(kmax); title = f"Acoustic dispersions, {κ=}, {order_x=}, {ppw=}"
anim = animation.FuncAnimation(fig, lambda i:frame_acoustic(κ,θs[i],dx,order_x,ks,title), frames=len(θs), repeat=False)
anim.save(savefig.dirName+to_filename(title)+'.mp4'); anim

### 11.2 Elastic

In [102]:
def frame_elastic(C,θ,dx,order_x,ks,title,mode,ωref):
    plt.clf()
    C = Hooke(C).rotate_by(θ).hooke    
    def af(x):return fd.as_field(x,ks.shape[1:])
    ω_exact,_ = Hooke(C).waves(ks,ρ)
    ω_Lebedev,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=order_x)
    ω_Selling_corr,_,_  = dispersion_SellingCorrelated(ks,af(ρ),af(C),dx,dt,order_x=order_x)
#    ω_Selling_stag,_,_ = dispersion_SellingStaggered2(ks,ρ,C,dx,dt,order_x=order_x)
    plt.contour(*ks,ω_exact[mode],levels=[ωref],colors='black')
    plt.contour(*ks,ω_Lebedev[mode],levels=[ωref],colors='blue')
    plt.contour(*ks,ω_Selling_corr[mode],levels=[ωref],colors='red')
#    plt.contour(*ks,ω_Selling_stag[mode],levels=[ωref],colors='green')
    plt.title(title)
    plt.legend(loc='lower right',handles=[Patch(color=color,label=label) for color,label in 
                    (('black','exact'),('blue','Lebedev'),('red','Selling_corr'))]) #,('green','Selling_stag'))])

def ωmin_k1(C,ρ,mode):
    """Minimal pulsation of given mode for a wave number with unit norm"""
    θs = np.linspace(0,π)
    ks = np.cos(θs),np.sin(θs)
    ω,_ = Hooke(C).waves(ks,ρ)
    return np.min(ω[mode])

In [103]:
ppw = 4; θ = π/7; order_x=2; kmax = 1.2; mode=1
C_name = "Stishovite"; C = Hooke.stishovite[0].extract_xz().hooke; dt=1e-3 # parameters
dx = 2*π/ppw; ks = mk_ks(kmax); ωref = ωmin_k1(C,ρ,mode); 
title = f"{C_name} {['shear','pressure'][mode]} dispersions, {order_x=}, {ppw=}"
fig = plt.figure(figsize=[6,6]); plt.axis('equal')
frame_elastic(C,θ,dx,order_x,ks,title,mode,ωref)

In [104]:
stishovite = Hooke.stishovite[0].extract_xz().hooke
mica = Hooke.mica[0].extract_xz().hooke
for (ppw,order_x,mode,C_name,C,kmax) in [
    (4,2,0,'Stishovite',stishovite,1.4),
    (4,2,1,'Stishovite',stishovite,1.4),
    (4,4,0,'Stishovite',stishovite,1.2),
    (4,4,1,'Stishovite',stishovite,1.2),
    (5,2,0,'Mica',mica,1.4),
    (5,2,1,'Mica',mica,1.4),
    (4,4,0,'Mica',mica,1.2),
    (4,4,1,'Mica',mica,1.2),
]:
    θ = 0.4; dt=1e-3; C = Hooke(C).rotate_by(θ).hooke #0.5
    dx = 2*π/ppw; ks = mk_ks(kmax); ωref = ωmin_k1(C,ρ,mode); 
    title = f"{C_name} {['shear','pressure'][mode]} dispersions, {order_x=}, {ppw=}"
    fig = plt.figure(figsize=[6,6]); plt.axis('equal')
    frame_elastic(C,θ,dx,order_x,ks,title,mode,ωref)
    savefig(fig,to_filename(title)+".png")
    plt.close(fig)

In [106]:
for (ppw,order_x,mode,C_name,C,kmax) in [
    (5,2,0,'Stishovite',stishovite,1.2),
    (5,2,1,'Stishovite',stishovite,1.4),
    (4,4,0,'Stishovite',stishovite,1.2),
    (4,4,1,'Stishovite',stishovite,1.2),
    (6,2,0,'Mica',mica,1.4),
    (5,2,1,'Mica',mica,1.4),
    (5,4,0,'Mica',mica,1.3),
    (4,4,1,'Mica',mica,1.2),
]:
    θs=np.linspace(0,π/2,30); dt=1e-3; #C = Hooke(C).rotate_by(θ).hooke #0.5
    dx = 2*π/ppw; ks = mk_ks(kmax); ωref = ωmin_k1(C,ρ,mode); 
    title = f"{C_name} {['shear','pressure'][mode]} dispersions, {order_x=}, {ppw=}"
    fig = plt.figure(figsize=[6,6]); plt.axis('equal')
    anim = animation.FuncAnimation(fig, lambda i:frame_elastic(C,θs[i],dx,order_x,ks,title,mode,ωref), frames=len(θs), repeat=False)
    anim.save(savefig.dirName+to_filename(title)+'.mp4')